In [ ]:
3. Statistical Foundation & Hypothesis Testing
3.1 Statistical Testing Framework
Why Statistical Testing in Level 3?
Level 2: Descriptive statistics - "Churn rate is 26.5%"
Level 3: Inferential statistics - "Is this difference real or random chance?"
Level 4: Predictive modeling - "Can we predict who will churn?"
3.2 Test Selection Decision Tree
def choose_statistical_test(data_type1, data_type2):
    """
    Systematic test selection based on data characteristics.
    """
    if data_type1 == 'categorical' and data_type2 == 'categorical':
        return 'chi_square'  # Test independence
    elif data_type1 == 'numerical' and data_type2 == 'categorical':
        # Check normality
        if check_normality(data):
            return 't_test'  # Parametric
        else:
            return 'mann_whitney_u'  # Non-parametric
    elif data_type1 == 'numerical' and data_type2 == 'numerical':
        return 'correlation'  # Pearson or Spearman

3.3 Statistical Testing Module (stats.py)
"""
Statistical Analysis Module - Level 3 Enhancement
LEARNING NOTES:
- Statistical validation prevents false conclusions
- Effect size matters more than p-values in business contexts
- Different tests for different data types and assumptions
"""

import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu
from statsmodels.stats.multitest import multipletests

def test_numerical_vs_churn(df, numerical_col, target_col='Churn'):
    """
    Test if numerical feature differs between churn groups.
    
    DECISION TREE:
    1. Test normality (Shapiro-Wilk)
    2. If normal: use t-test (parametric)
    3. If not normal: use Mann-Whitney U (non-parametric)
    4. Always report effect size (Cohen's d)
    """
    churned = df[df[target_col] == 'Yes'][numerical_col].dropna()
    retained = df[df[target_col] == 'No'][numerical_col].dropna()
    
    # Test normality
    _, p_norm_churned = stats.shapiro(churned.sample(min(100, len(churned))))
    _, p_norm_retained = stats.shapiro(retained.sample(min(100, len(retained))))
    
    is_normal = (p_norm_churned > 0.05) and (p_norm_retained > 0.05)
    
    # Choose appropriate test
    if is_normal:
        statistic, p_value = ttest_ind(churned, retained)
        test_used = 't-test'
    else:
        statistic, p_value = mannwhitneyu(churned, retained)
        test_used = 'Mann-Whitney U'
    
    # Calculate effect size (Cohen's d)
    pooled_std = np.sqrt(((len(churned)-1)*churned.std()**2 + 
                          (len(retained)-1)*retained.std()**2) / 
                         (len(churned) + len(retained) - 2))
    cohens_d = (churned.mean() - retained.mean()) / pooled_std
    
    # Interpret effect size
    if abs(cohens_d) < 0.2:
        effect_interpretation = "negligible"
    elif abs(cohens_d) < 0.5:
        effect_interpretation = "small"
    elif abs(cohens_d) < 0.8:
        effect_interpretation = "medium"
    else:
        effect_interpretation = "large"
    
    return {
        'test_used': test_used,
        'statistic': float(statistic),
        'p_value': float(p_value),
        'significant': p_value < 0.05,
        'churned_mean': float(churned.mean()),
        'retained_mean': float(retained.mean()),
        'cohens_d': float(cohens_d),
        'effect_size': effect_interpretation
    }




In [ ]:
# Statistical Approaches for Telco Customer Churn Analysis
## A Comprehensive Dissertation

---

## Table of Contents

1. [Executive Summary](#1-executive-summary)
2. [Introduction to Statistical Analysis in Churn Prediction](#2-introduction)
3. [Descriptive Statistics](#3-descriptive-statistics)
4. [Inferential Statistics](#4-inferential-statistics)
5. [Hypothesis Testing](#5-hypothesis-testing)
6. [Correlation and Association Analysis](#6-correlation-and-association-analysis)
7. [Distribution Analysis](#7-distribution-analysis)
8. [Time Series and Survival Analysis](#8-time-series-and-survival-analysis)
9. [Multivariate Statistical Techniques](#9-multivariate-statistical-techniques)
10. [Statistical Assumptions and Validation](#10-statistical-assumptions)
11. [Practical Implementation Guide](#11-practical-implementation)
12. [Case Studies and Applications](#12-case-studies)
13. [Conclusion](#13-conclusion)

---

## 1. Executive Summary

This dissertation provides a comprehensive guide to statistical approaches essential for analyzing customer churn in telecommunications. We cover 15+ statistical methods, their theoretical foundations, practical applications, and implementation in Python.

### Key Statistical Methods Covered:

- **Descriptive Statistics**: Central tendency, dispersion, distribution shapes
- **Hypothesis Testing**: t-tests, chi-square tests, ANOVA
- **Correlation Analysis**: Pearson, Spearman, point-biserial
- **Distribution Analysis**: Normality tests, Q-Q plots
- **Survival Analysis**: Kaplan-Meier, Cox regression
- **Multivariate Techniques**: PCA, factor analysis, cluster analysis

---

## 2. Introduction to Statistical Analysis in Churn Prediction

### 2.1 Why Statistics Matter in Churn Analysis

Statistical analysis forms the foundation of data-driven churn prediction by:

1. **Quantifying Relationships**: Measure strength between features and churn
2. **Testing Hypotheses**: Validate business assumptions scientifically
3. **Identifying Patterns**: Discover hidden trends in customer behavior
4. **Ensuring Validity**: Verify model assumptions and results
5. **Supporting Decisions**: Provide evidence-based recommendations

### 2.2 The Statistical Analysis Pipeline

```
Data Collection → Descriptive Statistics → Exploratory Analysis →
Hypothesis Testing → Model Building → Validation → Interpretation
```

### 2.3 Types of Variables in Churn Analysis

| Variable Type | Examples | Statistical Methods |
|---------------|----------|---------------------|
| **Binary** | Churn (Yes/No), Gender | Chi-square, logistic regression |
| **Nominal** | Contract type, Payment method | Chi-square, ANOVA |
| **Ordinal** | Satisfaction ratings, Tenure groups | Mann-Whitney U, Kruskal-Wallis |
| **Continuous** | Monthly charges, Tenure (months) | t-tests, correlation, regression |

---

## 3. Descriptive Statistics

Descriptive statistics summarize and describe the main features of your dataset.

### 3.1 Measures of Central Tendency

#### 3.1.1 Mean (Average)

**Definition**: Sum of all values divided by count

**Formula**: 
```
μ = (Σx) / n
```

**When to Use**:
- Continuous variables (tenure, charges)
- Normally distributed data
- No extreme outliers

**Python Implementation**:

```python
import pandas as pd
import numpy as np

# Calculate mean
mean_tenure = df['tenure'].mean()
mean_monthly_charges = df['MonthlyCharges'].mean()

# By churn status
df.groupby('Churn')['tenure'].mean()

# Interpretation
print(f"Average tenure: {mean_tenure:.2f} months")
print(f"Churned customers avg tenure: {df[df['Churn']=='Yes']['tenure'].mean():.2f}")
print(f"Retained customers avg tenure: {df[df['Churn']=='No']['tenure'].mean():.2f}")
```

**Interpretation for Churn**:
- If churned customers have lower mean tenure → New customers at risk
- If churned customers have higher mean charges → Price sensitivity issue

#### 3.1.2 Median

**Definition**: Middle value when data is sorted

**When to Use**:
- Skewed distributions
- Presence of outliers
- Ordinal data

**Python Implementation**:

```python
# Calculate median
median_tenure = df['tenure'].median()

# Compare mean vs median to detect skewness
print(f"Mean tenure: {df['tenure'].mean():.2f}")
print(f"Median tenure: {df['tenure'].median():.2f}")

# If mean > median: Right-skewed (long tail of high values)
# If mean < median: Left-skewed (long tail of low values)
```

#### 3.1.3 Mode

**Definition**: Most frequently occurring value

**When to Use**:
- Categorical variables
- Identify most common category

**Python Implementation**:

```python
# Most common contract type
mode_contract = df['Contract'].mode()[0]
print(f"Most common contract: {mode_contract}")

# Mode by churn status
df[df['Churn']=='Yes']['Contract'].mode()[0]
df[df['Churn']=='No']['Contract'].mode()[0]
```

### 3.2 Measures of Dispersion

#### 3.2.1 Standard Deviation

**Definition**: Average distance from the mean

**Formula**:
```
σ = sqrt(Σ(x - μ)² / n)
```

**Python Implementation**:

```python
# Calculate standard deviation
std_charges = df['MonthlyCharges'].std()

# Coefficient of Variation (CV) - standardized measure
cv = (std_charges / df['MonthlyCharges'].mean()) * 100
print(f"CV: {cv:.2f}% - Shows relative variability")

# Compare variability between groups
churned_std = df[df['Churn']=='Yes']['MonthlyCharges'].std()
retained_std = df[df['Churn']=='No']['MonthlyCharges'].std()

# Higher variability in churned group may indicate pricing issues
```

**Interpretation**:
- Low std dev: Homogeneous customer base
- High std dev: Diverse customer segments
- Compare between churn groups to identify differences

#### 3.2.2 Variance

**Definition**: Square of standard deviation

**Python Implementation**:

```python
variance = df['tenure'].var()

# Variance explained in churn analysis
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_scaled)
explained_variance_ratio = pca.explained_variance_ratio_

print("Variance explained by each component:")
for i, var in enumerate(explained_variance_ratio[:5]):
    print(f"PC{i+1}: {var*100:.2f}%")
```

#### 3.2.3 Range and Interquartile Range (IQR)

**Range**: Maximum - Minimum

**IQR**: Q3 - Q1 (middle 50% of data)

**Python Implementation**:

```python
# Calculate range
data_range = df['MonthlyCharges'].max() - df['MonthlyCharges'].min()

# Calculate IQR
Q1 = df['MonthlyCharges'].quantile(0.25)
Q3 = df['MonthlyCharges'].quantile(0.75)
IQR = Q3 - Q1

# Detect outliers using IQR method
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['MonthlyCharges'] < lower_bound) | 
              (df['MonthlyCharges'] > upper_bound)]

print(f"Number of outliers: {len(outliers)}")
print(f"Outlier percentage: {len(outliers)/len(df)*100:.2f}%")
```

### 3.3 Measures of Shape

#### 3.3.1 Skewness

**Definition**: Measure of asymmetry in distribution

**Interpretation**:
- Skewness = 0: Perfectly symmetric
- Skewness > 0: Right-skewed (tail on right)
- Skewness < 0: Left-skewed (tail on left)

**Python Implementation**:

```python
from scipy.stats import skew, kurtosis

# Calculate skewness
tenure_skew = skew(df['tenure'])
charges_skew = skew(df['MonthlyCharges'])

print(f"Tenure skewness: {tenure_skew:.3f}")
print(f"Monthly charges skewness: {charges_skew:.3f}")

# Interpret
if abs(tenure_skew) < 0.5:
    print("Tenure is approximately symmetric")
elif tenure_skew > 0:
    print("Tenure is right-skewed (many new customers)")
else:
    print("Tenure is left-skewed (many long-term customers)")

# Visualize
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['tenure'], bins=30, edgecolor='black')
axes[0].set_title(f'Tenure Distribution (Skewness: {tenure_skew:.2f})')
axes[0].axvline(df['tenure'].mean(), color='red', linestyle='--', label='Mean')
axes[0].axvline(df['tenure'].median(), color='green', linestyle='--', label='Median')
axes[0].legend()

axes[1].hist(df['MonthlyCharges'], bins=30, edgecolor='black')
axes[1].set_title(f'Monthly Charges (Skewness: {charges_skew:.2f})')
axes[1].axvline(df['MonthlyCharges'].mean(), color='red', linestyle='--', label='Mean')
axes[1].axvline(df['MonthlyCharges'].median(), color='green', linestyle='--', label='Median')
axes[1].legend()

plt.tight_layout()
plt.show()
```

#### 3.3.2 Kurtosis

**Definition**: Measure of "tailedness" or extreme values

**Interpretation**:
- Kurtosis = 3: Normal distribution (mesokurtic)
- Kurtosis > 3: Heavy tails, more outliers (leptokurtic)
- Kurtosis < 3: Light tails, fewer outliers (platykurtic)

**Python Implementation**:

```python
# Calculate excess kurtosis (subtract 3 for comparison to normal)
tenure_kurt = kurtosis(df['tenure'], fisher=True)  # fisher=True gives excess kurtosis

print(f"Tenure excess kurtosis: {tenure_kurt:.3f}")

if tenure_kurt > 0:
    print("→ More extreme values than normal distribution")
    print("→ May need robust statistical methods")
elif tenure_kurt < 0:
    print("→ Fewer extreme values than normal distribution")
    print("→ More uniform distribution")
```

### 3.4 Comprehensive Descriptive Statistics Summary

**Python Implementation**:

```python
def comprehensive_summary(df, column, churn_col='Churn'):
    """
    Generate comprehensive descriptive statistics for a column.
    """
    print(f"\n{'='*60}")
    print(f"COMPREHENSIVE STATISTICS: {column}")
    print(f"{'='*60}\n")
    
    # Overall statistics
    print("Overall Statistics:")
    print(f"  Count: {df[column].count()}")
    print(f"  Mean: {df[column].mean():.2f}")
    print(f"  Median: {df[column].median():.2f}")
    print(f"  Mode: {df[column].mode()[0] if len(df[column].mode()) > 0 else 'N/A'}")
    print(f"  Std Dev: {df[column].std():.2f}")
    print(f"  Variance: {df[column].var():.2f}")
    print(f"  Min: {df[column].min():.2f}")
    print(f"  Max: {df[column].max():.2f}")
    print(f"  Range: {df[column].max() - df[column].min():.2f}")
    
    # Percentiles
    print(f"\nPercentiles:")
    for p in [25, 50, 75, 90, 95, 99]:
        print(f"  {p}th: {df[column].quantile(p/100):.2f}")
    
    # Shape
    print(f"\nDistribution Shape:")
    print(f"  Skewness: {skew(df[column].dropna()):.3f}")
    print(f"  Kurtosis: {kurtosis(df[column].dropna(), fisher=True):.3f}")
    
    # By churn status
    print(f"\nBy Churn Status:")
    for churn_val in df[churn_col].unique():
        subset = df[df[churn_col]==churn_val][column]
        print(f"  {churn_val}:")
        print(f"    Mean: {subset.mean():.2f}")
        print(f"    Median: {subset.median():.2f}")
        print(f"    Std Dev: {subset.std():.2f}")
    
    # Missing values
    missing_pct = (df[column].isnull().sum() / len(df)) * 100
    print(f"\nData Quality:")
    print(f"  Missing: {df[column].isnull().sum()} ({missing_pct:.2f}%)")

# Usage
comprehensive_summary(df, 'tenure')
comprehensive_summary(df, 'MonthlyCharges')
```

---

## 4. Inferential Statistics

Inferential statistics allow us to make predictions and inferences about a population based on sample data.

### 4.1 Confidence Intervals

**Definition**: Range of values that likely contains the true population parameter

**Formula for Mean**:
```
CI = x̄ ± (t * (s / sqrt(n)))
```

Where:
- x̄ = sample mean
- t = t-value from t-distribution
- s = sample standard deviation
- n = sample size

**Python Implementation**:

```python
from scipy import stats

def calculate_confidence_interval(data, confidence=0.95):
    """
    Calculate confidence interval for mean.
    """
    n = len(data)
    mean = np.mean(data)
    std_err = stats.sem(data)  #

In [ ]:
# Statistical Approaches for Telco Customer Churn Analysis
## A Comprehensive Dissertation

---

## Table of Contents

1. [Executive Summary](#1-executive-summary)
2. [Introduction to Statistical Analysis in Churn Prediction](#2-introduction)
3. [Descriptive Statistics](#3-descriptive-statistics)
4. [Inferential Statistics](#4-inferential-statistics)
5. [Hypothesis Testing](#5-hypothesis-testing)
6. [Correlation and Association Analysis](#6-correlation-and-association-analysis)
7. [Distribution Analysis](#7-distribution-analysis)
8. [Time Series and Survival Analysis](#8-time-series-and-survival-analysis)
9. [Multivariate Statistical Techniques](#9-multivariate-statistical-techniques)
10. [Statistical Assumptions and Validation](#10-statistical-assumptions)
11. [Practical Implementation Guide](#11-practical-implementation)
12. [Case Studies and Applications](#12-case-studies)
13. [Conclusion](#13-conclusion)

---

## 1. Executive Summary

This dissertation provides a comprehensive guide to statistical approaches essential for analyzing customer churn in telecommunications. We cover 15+ statistical methods, their theoretical foundations, practical applications, and implementation in Python.

### Key Statistical Methods Covered:

- **Descriptive Statistics**: Central tendency, dispersion, distribution shapes
- **Hypothesis Testing**: t-tests, chi-square tests, ANOVA
- **Correlation Analysis**: Pearson, Spearman, point-biserial
- **Distribution Analysis**: Normality tests, Q-Q plots
- **Survival Analysis**: Kaplan-Meier, Cox regression
- **Multivariate Techniques**: PCA, factor analysis, cluster analysis

---

## 2. Introduction to Statistical Analysis in Churn Prediction

### 2.1 Why Statistics Matter in Churn Analysis

Statistical analysis forms the foundation of data-driven churn prediction by:

1. **Quantifying Relationships**: Measure strength between features and churn
2. **Testing Hypotheses**: Validate business assumptions scientifically
3. **Identifying Patterns**: Discover hidden trends in customer behavior
4. **Ensuring Validity**: Verify model assumptions and results
5. **Supporting Decisions**: Provide evidence-based recommendations

### 2.2 The Statistical Analysis Pipeline

```
Data Collection → Descriptive Statistics → Exploratory Analysis →
Hypothesis Testing → Model Building → Validation → Interpretation
```

### 2.3 Types of Variables in Churn Analysis

| Variable Type | Examples | Statistical Methods |
|---------------|----------|---------------------|
| **Binary** | Churn (Yes/No), Gender | Chi-square, logistic regression |
| **Nominal** | Contract type, Payment method | Chi-square, ANOVA |
| **Ordinal** | Satisfaction ratings, Tenure groups | Mann-Whitney U, Kruskal-Wallis |
| **Continuous** | Monthly charges, Tenure (months) | t-tests, correlation, regression |

---

## 3. Descriptive Statistics

Descriptive statistics summarize and describe the main features of your dataset.

### 3.1 Measures of Central Tendency

#### 3.1.1 Mean (Average)

**Definition**: Sum of all values divided by count

**Formula**: 
```
μ = (Σx) / n
```

**When to Use**:
- Continuous variables (tenure, charges)
- Normally distributed data
- No extreme outliers

**Python Implementation**:

```python
import pandas as pd
import numpy as np

# Calculate mean
mean_tenure = df['tenure'].mean()
mean_monthly_charges = df['MonthlyCharges'].mean()

# By churn status
df.groupby('Churn')['tenure'].mean()

# Interpretation
print(f"Average tenure: {mean_tenure:.2f} months")
print(f"Churned customers avg tenure: {df[df['Churn']=='Yes']['tenure'].mean():.2f}")
print(f"Retained customers avg tenure: {df[df['Churn']=='No']['tenure'].mean():.2f}")
```

**Interpretation for Churn**:
- If churned customers have lower mean tenure → New customers at risk
- If churned customers have higher mean charges → Price sensitivity issue

#### 3.1.2 Median

**Definition**: Middle value when data is sorted

**When to Use**:
- Skewed distributions
- Presence of outliers
- Ordinal data

**Python Implementation**:

```python
# Calculate median
median_tenure = df['tenure'].median()

# Compare mean vs median to detect skewness
print(f"Mean tenure: {df['tenure'].mean():.2f}")
print(f"Median tenure: {df['tenure'].median():.2f}")

# If mean > median: Right-skewed (long tail of high values)
# If mean < median: Left-skewed (long tail of low values)
```

#### 3.1.3 Mode

**Definition**: Most frequently occurring value

**When to Use**:
- Categorical variables
- Identify most common category

**Python Implementation**:

```python
# Most common contract type
mode_contract = df['Contract'].mode()[0]
print(f"Most common contract: {mode_contract}")

# Mode by churn status
df[df['Churn']=='Yes']['Contract'].mode()[0]
df[df['Churn']=='No']['Contract'].mode()[0]
```

### 3.2 Measures of Dispersion

#### 3.2.1 Standard Deviation

**Definition**: Average distance from the mean

**Formula**:
```
σ = sqrt(Σ(x - μ)² / n)
```

**Python Implementation**:

```python
# Calculate standard deviation
std_charges = df['MonthlyCharges'].std()

# Coefficient of Variation (CV) - standardized measure
cv = (std_charges / df['MonthlyCharges'].mean()) * 100
print(f"CV: {cv:.2f}% - Shows relative variability")

# Compare variability between groups
churned_std = df[df['Churn']=='Yes']['MonthlyCharges'].std()
retained_std = df[df['Churn']=='No']['MonthlyCharges'].std()

# Higher variability in churned group may indicate pricing issues
```

**Interpretation**:
- Low std dev: Homogeneous customer base
- High std dev: Diverse customer segments
- Compare between churn groups to identify differences

#### 3.2.2 Variance

**Definition**: Square of standard deviation

**Python Implementation**:

```python
variance = df['tenure'].var()

# Variance explained in churn analysis
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_scaled)
explained_variance_ratio = pca.explained_variance_ratio_

print("Variance explained by each component:")
for i, var in enumerate(explained_variance_ratio[:5]):
    print(f"PC{i+1}: {var*100:.2f}%")
```

#### 3.2.3 Range and Interquartile Range (IQR)

**Range**: Maximum - Minimum

**IQR**: Q3 - Q1 (middle 50% of data)

**Python Implementation**:

```python
# Calculate range
data_range = df['MonthlyCharges'].max() - df['MonthlyCharges'].min()

# Calculate IQR
Q1 = df['MonthlyCharges'].quantile(0.25)
Q3 = df['MonthlyCharges'].quantile(0.75)
IQR = Q3 - Q1

# Detect outliers using IQR method
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['MonthlyCharges'] < lower_bound) | 
              (df['MonthlyCharges'] > upper_bound)]

print(f"Number of outliers: {len(outliers)}")
print(f"Outlier percentage: {len(outliers)/len(df)*100:.2f}%")
```

### 3.3 Measures of Shape

#### 3.3.1 Skewness

**Definition**: Measure of asymmetry in distribution

**Interpretation**:
- Skewness = 0: Perfectly symmetric
- Skewness > 0: Right-skewed (tail on right)
- Skewness < 0: Left-skewed (tail on left)

**Python Implementation**:

```python
from scipy.stats import skew, kurtosis

# Calculate skewness
tenure_skew = skew(df['tenure'])
charges_skew = skew(df['MonthlyCharges'])

print(f"Tenure skewness: {tenure_skew:.3f}")
print(f"Monthly charges skewness: {charges_skew:.3f}")

# Interpret
if abs(tenure_skew) < 0.5:
    print("Tenure is approximately symmetric")
elif tenure_skew > 0:
    print("Tenure is right-skewed (many new customers)")
else:
    print("Tenure is left-skewed (many long-term customers)")

# Visualize
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['tenure'], bins=30, edgecolor='black')
axes[0].set_title(f'Tenure Distribution (Skewness: {tenure_skew:.2f})')
axes[0].axvline(df['tenure'].mean(), color='red', linestyle='--', label='Mean')
axes[0].axvline(df['tenure'].median(), color='green', linestyle='--', label='Median')
axes[0].legend()

axes[1].hist(df['MonthlyCharges'], bins=30, edgecolor='black')
axes[1].set_title(f'Monthly Charges (Skewness: {charges_skew:.2f})')
axes[1].axvline(df['MonthlyCharges'].mean(), color='red', linestyle='--', label='Mean')
axes[1].axvline(df['MonthlyCharges'].median(), color='green', linestyle='--', label='Median')
axes[1].legend()

plt.tight_layout()
plt.show()
```

#### 3.3.2 Kurtosis

**Definition**: Measure of "tailedness" or extreme values

**Interpretation**:
- Kurtosis = 3: Normal distribution (mesokurtic)
- Kurtosis > 3: Heavy tails, more outliers (leptokurtic)
- Kurtosis < 3: Light tails, fewer outliers (platykurtic)

**Python Implementation**:

```python
# Calculate excess kurtosis (subtract 3 for comparison to normal)
tenure_kurt = kurtosis(df['tenure'], fisher=True)  # fisher=True gives excess kurtosis

print(f"Tenure excess kurtosis: {tenure_kurt:.3f}")

if tenure_kurt > 0:
    print("→ More extreme values than normal distribution")
    print("→ May need robust statistical methods")
elif tenure_kurt < 0:
    print("→ Fewer extreme values than normal distribution")
    print("→ More uniform distribution")
```

### 3.4 Comprehensive Descriptive Statistics Summary

**Python Implementation**:

```python
def comprehensive_summary(df, column, churn_col='Churn'):
    """
    Generate comprehensive descriptive statistics for a column.
    """
    print(f"\n{'='*60}")
    print(f"COMPREHENSIVE STATISTICS: {column}")
    print(f"{'='*60}\n")
    
    # Overall statistics
    print("Overall Statistics:")
    print(f"  Count: {df[column].count()}")
    print(f"  Mean: {df[column].mean():.2f}")
    print(f"  Median: {df[column].median():.2f}")
    print(f"  Mode: {df[column].mode()[0] if len(df[column].mode()) > 0 else 'N/A'}")
    print(f"  Std Dev: {df[column].std():.2f}")
    print(f"  Variance: {df[column].var():.2f}")
    print(f"  Min: {df[column].min():.2f}")
    print(f"  Max: {df[column].max():.2f}")
    print(f"  Range: {df[column].max() - df[column].min():.2f}")
    
    # Percentiles
    print(f"\nPercentiles:")
    for p in [25, 50, 75, 90, 95, 99]:
        print(f"  {p}th: {df[column].quantile(p/100):.2f}")
    
    # Shape
    print(f"\nDistribution Shape:")
    print(f"  Skewness: {skew(df[column].dropna()):.3f}")
    print(f"  Kurtosis: {kurtosis(df[column].dropna(), fisher=True):.3f}")
    
    # By churn status
    print(f"\nBy Churn Status:")
    for churn_val in df[churn_col].unique():
        subset = df[df[churn_col]==churn_val][column]
        print(f"  {churn_val}:")
        print(f"    Mean: {subset.mean():.2f}")
        print(f"    Median: {subset.median():.2f}")
        print(f"    Std Dev: {subset.std():.2f}")
    
    # Missing values
    missing_pct = (df[column].isnull().sum() / len(df)) * 100
    print(f"\nData Quality:")
    print(f"  Missing: {df[column].isnull().sum()} ({missing_pct:.2f}%)")

# Usage
comprehensive_summary(df, 'tenure')
comprehensive_summary(df, 'MonthlyCharges')
```

---

## 4. Inferential Statistics

Inferential statistics allow us to make predictions and inferences about a population based on sample data.

### 4.1 Confidence Intervals

**Definition**: Range of values that likely contains the true population parameter

**Formula for Mean**:
```
CI = x̄ ± (t * (s / sqrt(n)))
```

Where:
- x̄ = sample mean
- t = t-value from t-distribution
- s = sample standard deviation
- n = sample size

**Python Implementation**:

```python
from scipy import stats

def calculate_confidence_interval(data, confidence=0.95):
    """
    Calculate confidence interval for mean.
    """
    n = len(data)
    mean = np.mean(data)
    std_err = stats.sem(data)  # Standard error of mean
    margin_error = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    
    ci_lower = mean - margin_error
    ci_upper = mean + margin_error
    
    return mean, ci_lower, ci_upper

# Example: Confidence interval for average tenure
churned_tenure = df[df['Churn']=='Yes']['tenure']
retained_tenure = df[df['Churn']=='No']['tenure']

mean_c, lower_c, upper_c = calculate_confidence_interval(churned_tenure)
mean_r, lower_r, upper_r = calculate_confidence_interval(retained_tenure)

print("Average Tenure with 95% Confidence Intervals:")
print(f"Churned: {mean_c:.2f} months [{lower_c:.2f}, {upper_c:.2f}]")
print(f"Retained: {mean_r:.2f} months [{lower_r:.2f}, {upper_r:.2f}]")

# Interpretation
if upper_c < lower_r:
    print("→ Churned customers have significantly lower tenure (no overlap)")
elif lower_c > upper_r:
    print("→ Churned customers have significantly higher tenure")
else:
    print("→ Confidence intervals overlap - difference may not be significant")
```

**Business Application**:
- Estimate true average monthly revenue from customers
- Predict churn rate with confidence bounds
- Compare segments with statistical rigor

### 4.2 Standard Error

**Definition**: Standard deviation of the sampling distribution

**Formula**:
```
SE = σ / sqrt(n)
```

**Python Implementation**:

```python
from scipy.stats import sem

# Calculate standard error for monthly charges
se_charges = sem(df['MonthlyCharges'])

print(f"Standard Error of Monthly Charges: ${se_charges:.2f}")
print(f"This means our sample mean is accurate within ±${se_charges:.2f}")

# Compare standard errors
se_churned = sem(df[df['Churn']=='Yes']['MonthlyCharges'])
se_retained = sem(df[df['Churn']=='No']['MonthlyCharges'])

print(f"\nSE for churned customers: ${se_churned:.2f}")
print(f"SE for retained customers: ${se_retained:.2f}")
```

---

## 5. Hypothesis Testing

Hypothesis testing is crucial for making data-driven decisions about churn drivers.

### 5.1 Framework for Hypothesis Testing

**Standard Process**:

1. **State Hypotheses**:
   - H₀ (Null): No difference/relationship exists
   - H₁ (Alternative): Difference/relationship exists

2. **Choose Significance Level (α)**:
   - Common: α = 0.05 (5% chance of Type I error)

3. **Calculate Test Statistic**

4. **Find p-value**

5. **Make Decision**:
   - If p-value < α: Reject H₀ (significant result)
   - If p-value ≥ α: Fail to reject H₀

### 5.2 Independent Samples t-Test

**Purpose**: Compare means of two independent groups

**Assumptions**:
- Both groups are normally distributed
- Equal variances (or use Welch's t-test)
- Independent observations

**When to Use in Churn Analysis**:
- Compare tenure between churned vs retained
- Compare charges between customer segments

**Python Implementation**:

```python
from scipy.stats import ttest_ind, levene, shapiro

def perform_t_test(group1, group2, group1_name, group2_name, 
                   variable_name, alpha=0.05):
    """
    Perform comprehensive independent t-test with assumption checks.
    """
    print(f"\n{'='*70}")
    print(f"INDEPENDENT T-TEST: {variable_name}")
    print(f"Comparing {group1_name} vs {group2_name}")
    print(f"{'='*70}\n")
    
    # 1. Check normality assumption
    print("1. Normality Tests (Shapiro-Wilk):")
    _, p_norm1 = shapiro(group1.sample(min(5000, len(group1))))  # Sample for large datasets
    _, p_norm2 = shapiro(group2.sample(min(5000, len(group2))))
    
    print(f"   {group1_name}: p-value = {p_norm1:.4f}")
    print(f"   {group2_name}: p-value = {p_norm2:.4f}")
    
    if p_norm1 > 0.05 and p_norm2 > 0.05:
        print("   ✓ Both groups appear normally distributed")
        normality_met = True
    else:
        print("   ⚠ At least one group deviates from normality")
        print("   → Consider using Mann-Whitney U test instead")
        normality_met = False
    
    # 2. Check equal variance assumption
    print("\n2. Equal Variance Test (Levene's Test):")
    _, p_var = levene(group1, group2)
    print(f"   p-value = {p_var:.4f}")
    
    if p_var > 0.05:
        print("   ✓ Variances are equal")
        equal_var = True
    else:
        print("   ⚠ Variances are unequal")
        print("   → Using Welch's t-test (doesn't assume equal variance)")
        equal_var = False
    
    # 3. Perform t-test
    print("\n3. T-Test Results:")
    t_stat, p_value = ttest_ind(group1, group2, equal_var=equal_var)
    
    print(f"   t-statistic: {t_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    print(f"   Significance level: {alpha}")
    
    # 4. Calculate effect size (Cohen's d)
    mean1, mean2 = group1.mean(), group2.mean()
    std1, std2 = group1.std(), group2.std()
    n1, n2 = len(group1), len(group2)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1-1)*std1**2 + (n2-1)*std2**2) / (n1+n2-2))
    cohens_d = (mean1 - mean2) / pooled_std
    
    print(f"\n4. Effect Size (Cohen's d): {cohens_d:.4f}")
    if abs(cohens_d) < 0.2:
        effect = "negligible"
    elif abs(cohens_d) < 0.5:
        effect = "small"
    elif abs(cohens_d) < 0.8:
        effect = "medium"
    else:
        effect = "large"
    print(f"   Effect size is {effect}")
    
    # 5. Interpretation
    print("\n5. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → Reject null hypothesis")
        print(f"   → {group1_name} and {group2_name} have different {variable_name}")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
        print(f"   → Fail to reject null hypothesis")
        print(f"   → Insufficient evidence of difference")
    
    # 6. Descriptive statistics
    print("\n6. Descriptive Statistics:")
    print(f"   {group1_name}: Mean = {mean1:.2f}, SD = {std1:.2f}, n = {n1}")
    print(f"   {group2_name}: Mean = {mean2:.2f}, SD = {std2:.2f}, n = {n2}")
    print(f"   Mean Difference: {abs(mean1 - mean2):.2f}")
    
    return {
        't_statistic': t_stat,
        'p_value': p_value,
        'cohens_d': cohens_d,
        'significant': p_value < alpha
    }

# Example: Compare tenure between churned and retained customers
churned = df[df['Churn']=='Yes']['tenure'].dropna()
retained = df[df['Churn']=='No']['tenure'].dropna()

results = perform_t_test(churned, retained, 
                         'Churned Customers', 'Retained Customers',
                         'Tenure (months)')
```

**Business Interpretation**:

```python
# If significant difference found:
if results['significant']:
    print("\n📊 BUSINESS INSIGHT:")
    print("Churned and retained customers have significantly different tenure.")
    print("→ Action: Focus retention efforts on specific tenure segments")
    print("→ Investigate: What happens at critical tenure milestones?")
```

### 5.3 Paired Samples t-Test

**Purpose**: Compare means of same group at two time points

**When to Use**:
- Before/after retention campaign
- Monthly charges across time periods

**Python Implementation**:

```python
from scipy.stats import ttest_rel

# Example: Compare customer satisfaction before and after intervention
# (hypothetical data)
satisfaction_before = df['satisfaction_before']
satisfaction_after = df['satisfaction_after']

t_stat, p_value = ttest_rel(satisfaction_before, satisfaction_after)

print(f"Paired t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("→ Intervention had significant effect on satisfaction")
```

### 5.4 Chi-Square Test for Independence

**Purpose**: Test relationship between two categorical variables

**When to Use in Churn Analysis**:
- Relationship between Contract type and Churn
- Relationship between Payment method and Churn
- Any categorical variable vs Churn

**Python Implementation**:

```python
from scipy.stats import chi2_contingency

def chi_square_test(df, var1, var2, alpha=0.05):
    """
    Perform chi-square test of independence.
    """
    print(f"\n{'='*70}")
    print(f"CHI-SQUARE TEST: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    # Create contingency table
    contingency_table = pd.crosstab(df[var1], df[var2])
    
    print("1. Contingency Table:")
    print(contingency_table)
    print()
    
    # Perform chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print("2. Test Results:")
    print(f"   Chi-square statistic: {chi2:.4f}")
    print(f"   p-value: {p_value:.4f}")
    print(f"   Degrees of freedom: {dof}")
    
    # Check expected frequencies assumption
    print("\n3. Assumption Check:")
    print("   Expected frequencies (should all be ≥ 5):")
    print(pd.DataFrame(expected, 
                       index=contingency_table.index,
                       columns=contingency_table.columns).round(2))
    
    min_expected = expected.min()
    if min_expected >= 5:
        print(f"   ✓ All expected frequencies ≥ 5 (min: {min_expected:.2f})")
        print("   ✓ Chi-square test is valid")
    else:
        print(f"   ⚠ Some expected frequencies < 5 (min: {min_expected:.2f})")
        print("   ⚠ Consider Fisher's exact test or combine categories")
    
    # Calculate effect size (Cramér's V)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape[0]-1, contingency_table.shape[1]-1)
    cramers_v = np.sqrt(chi2 / (n * min_dim))
    
    print(f"\n4. Effect Size (Cramér's V): {cramers_v:.4f}")
    if cramers_v < 0.1:
        effect = "negligible"
    elif cramers_v < 0.3:
        effect = "small"
    elif cramers_v < 0.5:
        effect = "medium"
    else:
        effect = "large"
    print(f"   Effect size is {effect}")
    
    # Interpretation
    print("\n5. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT ASSOCIATION (p < {alpha})")
        print(f"   → {var1} and {var2} are related")
        print(f"   → Variables are NOT independent")
    else:
        print(f"   ✗ NO SIGNIFICANT ASSOCIATION (p ≥ {alpha})")
        print(f"   → Insufficient evidence of relationship")
    
    # Calculate percentages for interpretation
    print("\n6. Percentage Breakdown:")
    pct_table = pd.crosstab(df[var1], df[var2], normalize='index') * 100
    print(pct_table.round(2))
    
    return {
        'chi2': chi2,
        'p_value': p_value,
        'cramers_v': cramers_v,
        'significant': p_value < alpha,
        'contingency_table': contingency_table
    }

# Example: Test relationship between Contract and Churn
result = chi_square_test(df, 'Contract', 'Churn')

# Business interpretation
if result['significant']:
    print("\n📊 BUSINESS INSIGHT:")
    print("Contract type is significantly related to churn.")
    print("→ Action: Analyze churn rates by contract type")
    print("→ Strategy: Incentivize longer contracts")
```

### 5.5 ANOVA (Analysis of Variance)

**Purpose**: Compare means across 3+ groups

**When to Use**:
- Compare charges across multiple contract types
- Compare tenure across service tiers

**Python Implementation**:

```python
from scipy.stats import f_oneway

def perform_anova(df, group_var, numeric_var, alpha=0.05):
    """
    Perform one-way ANOVA with post-hoc analysis.
    """
    print(f"\n{'='*70}")
    print(f"ONE-WAY ANOVA: {numeric_var} across {group_var}")
    print(f"{'='*70}\n")
    
    # Get groups
    groups = df[group_var].unique()
    group_data = [df[df[group_var]==g][numeric_var].dropna() for g in groups]
    
    # 1. Descriptive statistics
    print("1. Descriptive Statistics by Group:")
    for g, data in zip(groups, group_data):
        print(f"   {g}: Mean={data.mean():.2f}, SD={data.std():.2f}, n={len(data)}")
    
    # 2. Perform ANOVA
    print("\n2. ANOVA Results:")
    f_stat, p_value = f_oneway(*group_data)
    
    print(f"   F-statistic: {f_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # 3. Effect size (eta-squared)
    # Calculate between-group and total sum of squares
    grand_mean = df[numeric_var].mean()
    ss_between = sum([len(data) * (data.mean() - grand_mean)**2 
                      for data in group_data])
    ss_total = sum([(x - grand_mean)**2 for data in group_data for x in data])
    eta_squared = ss_between / ss_total
    
    print(f"\n3. Effect Size (η²): {eta_squared:.4f}")
    print(f"   {eta_squared*100:.2f}% of variance explained by {group_var}")
    
    # 4. Interpretation
    print("\n4. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → At least one group differs significantly")
        print(f"   → Recommend post-hoc tests (Tukey HSD)")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
        print(f"   → All groups have similar means")
    
    # 5. Post-hoc test (Tukey HSD) if significant
    if p_value < alpha:
        from statsmodels.stats.multicomp import pairwise_tukeyhsd
        
        print("\n5. Post-Hoc Analysis (Tukey HSD):")
        tukey = pairwise_tukeyhsd(df[numeric_var], df[group_var], alpha=alpha)
        print(tukey)
    
    return {
        'f_statistic': f_stat,
        'p_value': p_value,
        'eta_squared': eta_squared,
        'significant': p_value < alpha
    }

# Example: Compare monthly charges across contract types
result = perform_anova(df, 'Contract', 'MonthlyCharges')
```

### 5.6 Mann-Whitney U Test (Non-Parametric Alternative)

**Purpose**: Compare distributions of two groups without normality assumption

**When to Use**:
- Data is not normally distributed
- Ordinal data
- Small sample sizes

**Python Implementation**:

```python
from scipy.stats import mannwhitneyu

def mann_whitney_test(group1, group2, group1_name, group2_name, 
                      variable_name, alpha=0.05):
    """
    Perform Mann-Whitney U test (non-parametric alternative to t-test).
    """
    print(f"\n{'='*70}")
    print(f"MANN-WHITNEY U TEST: {variable_name}")
    print(f"Comparing {group1_name} vs {group2_name}")
    print(f"{'='*70}\n")
    
    # Perform test
    u_stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
    
    print("1. Test Results:")
    print(f"   U-statistic: {u_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Calculate effect size (rank-biserial correlation)
    n1, n2 = len(group1), len(group2)
    r = 1 - (2*u_stat) / (n1 * n2)  # rank-biserial correlation
    
    print(f"\n2. Effect Size (rank-biserial r): {r:.4f}")
    
    # Interpretation
    print("\n3. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → Distributions differ significantly")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
    
    # Medians for interpretation
    print("\n4. Median Comparison:")
    print(f"   {group1_name}: Median = {group1.median():.2f}")
    print(f"   {group2_name}: Median = {group2.median():.2f}")
    
    return {
        'u_statistic': u_stat,
        'p_value': p_value,
        'effect_size': r,
        'significant': p_value < alpha
    }

# Example: When data is not normally distributed
churned_charges = df[df['Churn']=='Yes']['MonthlyCharges'].dropna()
retained_charges = df[df['Churn']=='No']['MonthlyCharges'].dropna()

result = mann_whitney_test(churned_charges, retained_charges,
                           'Churned', 'Retained', 'Monthly Charges')
```

---

## 6. Correlation and Association Analysis

Understanding relationships between variables is crucial for feature selection and model building.

### 6.1 Pearson Correlation

**Purpose**: Measure linear relationship between two continuous variables

**Formula**:
```
r = Σ((x - x̄)(y - ȳ)) / sqrt(Σ(x - x̄)² × Σ(y - ȳ)²)
```

**Interpretation**:
- r = 1: Perfect positive correlation
- r = 0: No linear correlation
- r = -1: Perfect negative correlation
- |r| < 0.3: Weak
- 0.3 ≤ |r| < 0.7: Moderate
- |r| ≥ 0.7: Strong

**Python Implementation**:

```python
from scipy.stats import pearsonr

def pearson_correlation_analysis(df, var1, var2, alpha=0.05):
    """
    Comprehensive Pearson correlation analysis.
    """
    print(f"\n{'='*70}")
    print(f"PEARSON CORRELATION: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    # Remove missing values
    data = df[[var1, var2]].dropna()
    
    # Calculate correlation
    r, p_value = pearsonr(data[var1], data[var2])
    
    print("1. Correlation Results:")
    print(f"   Pearson r: {r:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Interpret strength
    if abs(r) < 0.3:
        strength = "weak"
    elif abs(r) < 0.7:
        strength = "moderate"
    else:
        strength = "strong"
    
    direction = "positive" if r > 0 else "negative"
    
    print(f"   Strength: {strength} {direction} correlation")
    
    # Calculate coefficient of determination
    r_squared = r ** 2
    print(f"\n2. Coefficient of Determination (r²): {r_squared:.4f}")
    print(f"   {r_squared*100:.2f}% of variance in {var2} explained by {var1}")
    
    # Statistical significance
    print("\n3. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT CORRELATION (p < {alpha})")
        print(f"   → Relationship is statistically significant")
    else:
        print(f"   ✗ NO SIGNIFICANT CORRELATION (p ≥ {alpha})")
    
    # Visualization
    plt.figure(figsize=(10, 6))
    plt.scatter(data[var1], data[var2], alpha=0.5)
    
    # Add regression line
    z = np.polyfit(data[var1], data[var2], 1)
    p = np.poly1d(z)
    plt.plot(data[var1], p(data[var1]), "r--", linewidth=2, label='Regression line')
    
    plt.xlabel(var1, fontsize=12)
    plt.ylabel(var2, fontsize=12)
    plt.title(f'{var1} vs {var2}\n(r = {r:.3f}, p = {p_value:.4f})', 
              fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {'r': r, 'p_value': p_value, 'r_squared': r_squared}

# Example: Correlation between tenure and total charges
result = pearson_correlation_analysis(df, 'tenure', 'TotalCharges')
```

### 6.2 Spearman Correlation

**Purpose**: Measure monotonic relationship (not necessarily linear)

**When to Use**:
- Ordinal variables
- Non-linear relationships
- Non-normal distributions
- Outliers present

**Python Implementation**:

```python
from scipy.stats import spearmanr

def spearman_correlation_analysis(df, var1, var2, alpha=0.05):
    """
    Spearman rank correlation analysis.
    """
    print(f"\n{'='*70}")
    print(f"SPEARMAN CORRELATION: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    data = df[[var1, var2]].dropna()
    
    # Calculate Spearman correlation
    rho, p_value = spearmanr(data[var1], data[var2])
    
    print("1. Correlation Results:")
    print(f"   Spearman ρ (rho): {rho:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Compare with Pearson
    r_pearson, _ = pearsonr(data[var1], data[var2])
    print(f"\n2. Comparison:")
    print(f"   Pearson r:  {r_pearson:.4f}")
    print(f"   Spearman ρ: {rho:.4f}")
    print(f"   Difference: {abs(r_pearson - rho):.4f}")
    
    if abs(r_pearson - rho) > 0.1:
        print("   ⚠ Large difference suggests non-linear relationship")
    else:
        print("   ✓ Similar values suggest linear relationship")
    
    return {'rho': rho, 'p_value': p_value}

# Example
result = spearman_correlation_analysis(df, 'tenure', 'MonthlyCharges')
```

### 6.3 Point-Biserial Correlation

**Purpose**: Correlation between continuous and binary variable

**When to Use**:
- Relationship between numeric variable and Churn (binary)

**Python Implementation**:

```python
from scipy.stats import pointbiserialr

def point_biserial_analysis(df, continuous_var, binary_var, alpha=0.05):
    """
    Point-biserial correlation for continuous vs binary variable.
    """
    print(f"\n{'='*70}")
    print(f"POINT-BISERIAL CORRELATION")
    print(f"{continuous_var} vs {binary_var}")
    print(f"{'='*70}\n")
    
    # Ensure binary variable is 0/1
    data = df[[continuous_var, binary_var]].dropna()
    if data[binary_var].dtype == 'object':
        binary_map = {data[binary_var].unique()[0]: 0,
                     data[binary_var].unique()[1]: 1}
        data[binary_var] = data[binary_var].map(binary_map)
    
    # Calculate correlation
    r_pb, p_value = pointbiserialr(data[binary_var], data[continuous_var])
    
    print("1. Correlation Results:")
    print(f"   Point-biserial r: {r_pb:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    print(f"\n2. Interpretation:")
    if r_pb > 0:
        print(f"   Positive correlation: Higher {continuous_var} → More likely {binary_var}=1")
    else:
        print(f"   Negative correlation: Higher {continuous_var} → More likely {binary_var}=0")
    
    if p_value < alpha:
        print(f"\n3. Conclusion: SIGNIFICANT relationship (p < {alpha})")
    else:
        print(f"\n3. Conclusion: NO significant relationship (p ≥ {alpha})")
    
    return {'r_pb': r_pb, 'p_value': p_value}

# Example: Tenure vs Churn
result = point_biserial_analysis(df, 'tenure', 'Churn')
```

### 6.4 Correlation Matrix and Heatmap

**Purpose**: Visualize all pairwise correlations

**Python Implementation**:

```python
def comprehensive_correlation_analysis(df, method='pearson'):
    """
    Create comprehensive correlation matrix with visualization.
    """
    # Select numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Calculate correlation matrix
    if method == 'pearson':
        corr_matrix = numeric_df.corr()
    elif method == 'spearman':
        corr_matrix = numeric_df.corr(method='spearman')
    
    # Create visualization
    plt.figure(figsize=(12, 10))
    
    # Create mask for upper triangle
    mask = np.triu(np.ones_like(corr_matrix), k=1)
    
    # Plot heatmap
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', 
                center=0, fmt='.2f', square=True, linewidths=1,
                cbar_kws={"shrink": 0.8})
    
    plt.title(f'{method.capitalize()} Correlation Matrix', 
              fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Find strongest correlations with target (if Churn exists)
    if 'Churn' in corr_matrix.columns:
        print("\nStrongest Correlations with Churn:")
        churn_corr = corr_matrix['Churn'].abs().sort_values(ascending=False)
        print(churn_corr[1:11])  # Top 10, excluding Churn itself
    
    return corr_matrix

# Usage
corr_matrix = comprehensive_correlation_analysis(df, method='pearson')
```

---

## 7. Distribution Analysis

Understanding data distributions is critical for choosing appropriate statistical tests and models.

### 7.1 Normality Tests

#### 7.1.1 Shapiro-Wilk Test

**Purpose**: Test if data comes from normal distribution

**Python Implementation**:

```python
from scipy.stats import shapiro

def test_normality(data, variable_name, alpha=0.05):
    """
    Comprehensive normality testing.
    """
    print(f"\n{'='*70}")
    print(f"NORMALITY TEST: {variable_name}")
    print(f"{'='*70}\n")
    
    # Shapiro-Wilk test
    stat, p_value = shapiro(data.sample(min(5000, len(data))))  # Sample for large datasets
    
    print("1. Shapiro-Wilk Test:")
    print(f"   Statistic: {stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    if p_value > alpha:
        print(f"   ✓ Data appears normally distributed (p > {alpha})")
        normal = True
    else:
        print(f"   ✗ Data deviates from normal distribution (p ≤ {alpha})")
        normal = False
    
    # Visual checks
    

In [ ]:
    # Visual checks
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Histogram with normal curve overlay
    axes[0].hist(data, bins=30, density=True, alpha=0.7, edgecolor='black')
    mu, sigma = data.mean(), data.std()
    x = np.linspace(data.min(), data.max(), 100)
    axes[0].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal distribution')
    axes[0].set_title('Histogram with Normal Curve', fontweight='bold')
    axes[0].set_xlabel(variable_name)
    axes[0].legend()
    
    # Q-Q plot
    stats.probplot(data, dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    # Box plot
    axes[2].boxplot(data, vert=True)
    axes[2].set_title('Box Plot', fontweight='bold')
    axes[2].set_ylabel(variable_name)
    axes[2].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n2. Visual Interpretation:")
    print("   - Histogram: Should resemble bell curve")
    print("   - Q-Q Plot: Points should fall on diagonal line")
    print("   - Box Plot: Should be roughly symmetric")
    
    return {'statistic': stat, 'p_value': p_value, 'normal': normal}

# Example
result = test_normality(df['tenure'], 'Tenure (months)')
result = test_normality(df['MonthlyCharges'], 'Monthly Charges ($)')
```

#### 7.1.2 Kolmogorov-Smirnov Test

**Purpose**: Alternative normality test, better for larger samples

**Python Implementation**:

```python
from scipy.stats import kstest

def ks_normality_test(data, variable_name, alpha=0.05):
    """
    Kolmogorov-Smirnov test for normality.
    """
    # Standardize data
    data_std = (data - data.mean()) / data.std()
    
    # Perform KS test
    stat, p_value = kstest(data_std, 'norm')
    
    print(f"\nKolmogorov-Smirnov Test for {variable_name}:")
    print(f"  Statistic: {stat:.4f}")
    print(f"  p-value: {p_value:.4f}")
    
    if p_value > alpha:
        print(f"  ✓ Data appears normally distributed")
    else:
        print(f"  ✗ Data deviates from normality")
    
    return stat, p_value

ks_normality_test(df['tenure'], 'Tenure')
```

### 7.2 Skewness and Kurtosis Tests

**Python Implementation**:

```python
from scipy.stats import skewtest, kurtosistest

def distribution_shape_tests(data, variable_name, alpha=0.05):
    """
    Test skewness and kurtosis significance.
    """
    print(f"\n{'='*60}")
    print(f"DISTRIBUTION SHAPE TESTS: {variable_name}")
    print(f"{'='*60}\n")
    
    # Calculate skewness
    skew_val = skew(data)
    skew_stat, skew_p = skewtest(data)
    
    print("1. Skewness Test:")
    print(f"   Skewness: {skew_val:.4f}")
    print(f"   Test statistic: {skew_stat:.4f}")
    print(f"   p-value: {skew_p:.4f}")
    
    if skew_p < alpha:
        if skew_val > 0:
            print("   ✓ Significantly right-skewed")
        else:
            print("   ✓ Significantly left-skewed")
    else:
        print("   → Skewness not significantly different from 0")
    
    # Calculate kurtosis
    kurt_val = kurtosis(data, fisher=True)
    kurt_stat, kurt_p = kurtosistest(data)
    
    print("\n2. Kurtosis Test:")
    print(f"   Excess kurtosis: {kurt_val:.4f}")
    print(f"   Test statistic: {kurt_stat:.4f}")
    print(f"   p-value: {kurt_p:.4f}")
    
    if kurt_p < alpha:
        if kurt_val > 0:
            print("   ✓ Significantly leptokurtic (heavy tails)")
        else:
            print("   ✓ Significantly platykurtic (light tails)")
    else:
        print("   → Kurtosis not significantly different from normal")

distribution_shape_tests(df['tenure'], 'Tenure')
```

---

## 8. Time Series and Survival Analysis

### 8.1 Survival Analysis (Kaplan-Meier)

**Purpose**: Analyze time until event (churn) occurs

**When to Use**:
- Understand customer lifetime
- Identify critical time periods for churn
- Compare survival across customer segments

**Python Implementation**:

```python
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

def survival_analysis(df, duration_col, event_col, group_col=None):
    """
    Comprehensive survival analysis for churn.
    """
    print(f"\n{'='*70}")
    print("SURVIVAL ANALYSIS (KAPLAN-MEIER)")
    print(f"{'='*70}\n")
    
    # Initialize Kaplan-Meier fitter
    kmf = KaplanMeierFitter()
    
    if group_col is None:
        # Overall survival curve
        kmf.fit(df[duration_col], df[event_col], label='All Customers')
        
        print("Overall Survival Statistics:")
        print(f"  Median survival time: {kmf.median_survival_time_:.2f} months")
        
        # Plot
        plt.figure(figsize=(12, 6))
        kmf.plot_survival_function()
        plt.title('Customer Survival Curve', fontsize=14, fontweight='bold')
        plt.xlabel('Tenure (months)')
        plt.ylabel('Survival Probability')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        
    else:
        # Survival by groups
        print(f"Survival Analysis by {group_col}:\n")
        
        plt.figure(figsize=(12, 6))
        
        groups = df[group_col].unique()
        group_data = []
        
        for group in groups:
            mask = df[group_col] == group
            group_subset = df[mask]
            
            kmf.fit(group_subset[duration_col], 
                   group_subset[event_col],
                   label=str(group))
            
            kmf.plot_survival_function()
            
            print(f"  {group}:")
            print(f"    Median survival: {kmf.median_survival_time_:.2f} months")
            print(f"    1-year survival: {kmf.survival_function_at_times(12).values[0]:.2%}")
            print(f"    2-year survival: {kmf.survival_function_at_times(24).values[0]:.2%}\n")
            
            group_data.append((group_subset[duration_col], group_subset[event_col]))
        
        plt.title(f'Survival Curves by {group_col}', fontsize=14, fontweight='bold')
        plt.xlabel('Tenure (months)')
        plt.ylabel('Survival Probability')
        plt.grid(alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        # Log-rank test if 2 groups
        if len(groups) == 2:
            result = logrank_test(group_data[0][0], group_data[1][0],
                                 group_data[0][1], group_data[1][1])
            
            print(f"Log-Rank Test:")
            print(f"  Test statistic: {result.test_statistic:.4f}")
            print(f"  p-value: {result.p_value:.4f}")
            
            if result.p_value < 0.05:
                print(f"  ✓ Survival curves are significantly different")
            else:
                print(f"  ✗ No significant difference between groups")

# Prepare data (tenure as duration, Churn as event)
df_survival = df.copy()
df_survival['Churn_binary'] = (df_survival['Churn'] == 'Yes').astype(int)

# Overall survival
survival_analysis(df_survival, 'tenure', 'Churn_binary')

# Survival by contract type
survival_analysis(df_survival, 'tenure', 'Churn_binary', 'Contract')
```

**Business Interpretation**:

```python
print("\n📊 BUSINESS INSIGHTS FROM SURVIVAL ANALYSIS:")
print("1. Median survival time tells us typical customer lifetime")
print("2. Steep drops indicate critical churn periods")
print("3. Compare curves across segments to prioritize interventions")
print("4. 1-year survival rate = retention rate at 12 months")
```

### 8.2 Cox Proportional Hazards Model

**Purpose**: Identify factors affecting time to churn

**Python Implementation**:

```python
from lifelines import CoxPHFitter

def cox_regression_analysis(df, duration_col, event_col, covariates):
    """
    Cox proportional hazards model for churn.
    """
    print(f"\n{'='*70}")
    print("COX PROPORTIONAL HAZARDS MODEL")
    print(f"{'='*70}\n")
    
    # Prepare data
    analysis_df = df[[duration_col, event_col] + covariates].dropna()
    
    # Fit model
    cph = CoxPHFitter()
    cph.fit(analysis_df, duration_col=duration_col, event_col=event_col)
    
    # Display results
    print("Model Summary:")
    print(cph.summary)
    
    print("\n\nInterpretation of Hazard Ratios:")
    print("(exp(coef) = Hazard Ratio)")
    print("-" * 60)
    
    for var in covariates:
        coef = cph.params_[var]
        hr = np.exp(coef)
        p_val = cph.summary.loc[var, 'p']
        
        print(f"\n{var}:")
        print(f"  Hazard Ratio: {hr:.4f}")
        
        if hr > 1:
            print(f"  → Increases churn risk by {(hr-1)*100:.1f}%")
        else:
            print(f"  → Decreases churn risk by {(1-hr)*100:.1f}%")
        
        if p_val < 0.05:
            print(f"  ✓ Statistically significant (p={p_val:.4f})")
        else:
            print(f"  ✗ Not significant (p={p_val:.4f})")
    
    # Plot
    cph.plot()
    plt.title('Hazard Ratios with 95% CI', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return cph

# Example with numeric predictors
covariates = ['MonthlyCharges', 'SeniorCitizen', 'Partner', 'Dependents']
cph_model = cox_regression_analysis(df_survival, 'tenure', 'Churn_binary', covariates)
```

---

## 9. Multivariate Statistical Techniques

### 9.1 Principal Component Analysis (PCA)

**Purpose**: Reduce dimensionality while preserving variance

**When to Use**:
- Many correlated features
- Visualization of high-dimensional data
- Feature extraction

**Python Implementation**:

```python
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def comprehensive_pca_analysis(df, n_components=None):
    """
    Complete PCA analysis with interpretation.
    """
    print(f"\n{'='*70}")
    print("PRINCIPAL COMPONENT ANALYSIS (PCA)")
    print(f"{'='*70}\n")
    
    # Select numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[numeric_cols].dropna()
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Fit PCA
    if n_components is None:
        n_components = min(len(numeric_cols), len(X))
    
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # 1. Variance explained
    print("1. Variance Explained:")
    cumsum_var = np.cumsum(pca.explained_variance_ratio_)
    
    for i, (var, cumvar) in enumerate(zip(pca.explained_variance_ratio_, cumsum_var)):
        print(f"   PC{i+1}: {var*100:.2f}% (Cumulative: {cumvar*100:.2f}%)")
        if cumvar >= 0.95:
            print(f"   → 95% variance explained with {i+1} components")
            break
    
    # 2. Scree plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Variance explained
    axes[0].bar(range(1, len(pca.explained_variance_ratio_)+1), 
                pca.explained_variance_ratio_, alpha=0.7)
    axes[0].plot(range(1, len(cumsum_var)+1), cumsum_var, 'r-o', linewidth=2)
    axes[0].set_xlabel('Principal Component', fontweight='bold')
    axes[0].set_ylabel('Variance Explained', fontweight='bold')
    axes[0].set_title('Scree Plot', fontweight='bold')
    axes[0].axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Biplot (first 2 components)
    axes[1].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
    axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontweight='bold')
    axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontweight='bold')
    axes[1].set_title('PCA Biplot (First 2 Components)', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 3. Component loadings
    print("\n2. Top Feature Loadings for First 3 Components:")
    loadings = pd.DataFrame(
        pca.components_.T,
        columns=[f'PC{i+1}' for i in range(pca.n_components_)],
        index=numeric_cols
    )
    
    for i in range(min(3, pca.n_components_)):
        print(f"\n   PC{i+1} - Top Features:")
        top_features = loadings[f'PC{i+1}'].abs().sort_values(ascending=False).head(5)
        for feature, loading in top_features.items():
            actual_loading = loadings.loc[feature, f'PC{i+1}']
            print(f"     {feature}: {actual_loading:.3f}")
    
    return pca, X_pca, loadings

# Example
pca_model, X_transformed, loadings = comprehensive_pca_analysis(df)
```

### 9.2 Factor Analysis

**Purpose**: Identify latent factors underlying observed variables

**Python Implementation**:

```python
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

def factor_analysis_comprehensive(df, n_factors=3):
    """
    Comprehensive factor analysis.
    """
    print(f"\n{'='*70}")
    print("FACTOR ANALYSIS")
    print(f"{'='*70}\n")
    
    # Select numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[numeric_cols].dropna()
    
    # 1. Test suitability for factor analysis
    print("1. Suitability Tests:")
    
    # KMO Test
    kmo_all, kmo_model = calculate_kmo(X)
    print(f"   Kaiser-Meyer-Olkin (KMO) Test: {kmo_model:.3f}")
    if kmo_model >= 0.6:
        print("   ✓ Data suitable for factor analysis (KMO > 0.6)")
    else:
        print("   ⚠ Data may not be suitable (KMO < 0.6)")
    
    # Bartlett's Test
    chi_square, p_value = calculate_bartlett_sphericity(X)
    print(f"\n   Bartlett's Test of Sphericity:")
    print(f"     Chi-square: {chi_square:.2f}")
    print(f"     p-value: {p_value:.4f}")
    if p_value < 0.05:
        print("   ✓ Variables are correlated (suitable for FA)")
    else:
        print("   ⚠ Variables may not be sufficiently correlated")
    
    # 2. Fit factor analysis
    print(f"\n2. Fitting {n_factors}-Factor Model:")
    
    fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax')
    fa.fit(X)
    
    # Get factor loadings
    loadings = pd.DataFrame(
        fa.loadings_,
        index=numeric_cols,
        columns=[f'Factor{i+1}' for i in range(n_factors)]
    )
    
    print("\n   Factor Loadings:")
    print(loadings.round(3))
    
    # 3. Variance explained
    variance = fa.get_factor_variance()
    
    print("\n3. Variance Explained:")
    print(f"   Proportional variance: {variance[1]}")
    print(f"   Cumulative variance: {variance[2]}")
    
    # 4. Interpret factors
    print("\n4. Factor Interpretation:")
    for i in range(n_factors):
        print(f"\n   Factor {i+1} - Top Loaded Variables:")
        top_vars = loadings[f'Factor{i+1}'].abs().sort_values(ascending=False).head(5)
        for var, loading in top_vars.items():
            actual = loadings.loc[var, f'Factor{i+1}']
            print(f"     {var}: {actual:.3f}")
    
    return fa, loadings

# Example
fa_model, factor_loadings = factor_analysis_comprehensive(df, n_factors=3)
```

### 9.3 Cluster Analysis

**Purpose**: Group similar customers together

**Python Implementation**:

```python
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

def cluster_analysis(df, n_clusters_range=range(2, 11)):
    """
    K-means clustering with optimal cluster selection.
    """
    print(f"\n{'='*70}")
    print("CLUSTER ANALYSIS (K-MEANS)")
    print(f"{'='*70}\n")
    
    # Prepare data
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[numeric_cols].dropna()
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 1. Determine optimal number of clusters
    print("1. Finding Optimal Number of Clusters:")
    
    inertias = []
    silhouette_scores = []
    
    for k in n_clusters_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    
    # Plot elbow curve and silhouette scores
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Elbow method
    axes[0].plot(n_clusters_range, inertias, 'bo-', linewidth=2)
    axes[0].set_xlabel('Number of Clusters', fontweight='bold')
    axes[0].set_ylabel('Inertia', fontweight='bold')
    axes[0].set_title('Elbow Method', fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Silhouette score
    axes[1].plot(n_clusters_range, silhouette_scores, 'ro-', linewidth=2)
    axes[1].set_xlabel('Number of Clusters', fontweight='bold')
    axes[1].set_ylabel('Silhouette Score', fontweight='bold')
    axes[1].set_title('Silhouette Analysis', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Select optimal k (highest silhouette score)
    optimal_k = n_clusters_range[np.argmax(silhouette_scores)]
    print(f"\n   Optimal number of clusters: {optimal_k}")
    print(f"   Best silhouette score: {max(silhouette_scores):.3f}")
    
    # 2. Fit final model
    print(f"\n2. Fitting {optimal_k}-Cluster Model:")
    
    kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    clusters = kmeans_final.fit_predict(X_scaled)
    
    # Add clusters to dataframe
    df_clustered = X.copy()
    df_clustered['Cluster'] = clusters
    
    # 3. Analyze clusters
    print(f"\n3. Cluster Profiles:")
    
    for i in range(optimal_k):
        cluster_data = df_clustered[df_clustered['Cluster'] == i]
        print(f"\n   Cluster {i} (n={len(cluster_data)}):")
        print(f"     Mean values:")
        for col in numeric_cols[:5]:  # Show top 5 features
            print(f"       {col}: {cluster_data[col].mean():.2f}")
    
    # 4. Visualize clusters (2D PCA)
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.6)
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontweight='bold')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontweight='bold')
    plt.title('Customer Segments (K-Means Clustering)', fontweight='bold', fontsize=14)
    plt.colorbar(scatter, label='Cluster')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return kmeans_final, clusters, df_clustered

# Example
kmeans_model, cluster_labels, df_with_clusters = cluster_analysis(df)
```

---

## 10. Statistical Assumptions and Validation

### 10.1 Checking Linear Regression Assumptions

**Python Implementation**:

```python
from sklearn.linear_model import LinearRegression
from scipy.stats import jarque_bera

def check_regression_assumptions(X, y, feature_names):
    """
    Comprehensive check of linear regression assumptions.
    """
    print(f"\n{'='*70}")
    print("LINEAR REGRESSION ASSUMPTIONS CHECK")
    print(f"{'='*70}\n")
    
    # Fit model
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    residuals = y - y_pred
    
    # 1. Linearity
    print("1. LINEARITY (Residuals vs Fitted Values):")
    plt.figure(figsize=(10, 6))
    plt.scatter(y_pred, residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
    plt.xlabel('Fitted Values', fontweight='bold')
    plt.ylabel('Residuals', fontweight='bold')
    plt.title('Residual Plot', fontweight='bold')
    plt.grid(alpha=0.3)
    plt.show()
    print("   → Pattern should be random scatter around zero")
    print("   → Funnel shape indicates heteroscedasticity")
    print("   → Curve indicates non-linearity")
    
    # 2. Normality of residuals
    print("\n2. NORMALITY OF RESIDUALS:")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histogram
    axes[0].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
    axes[0].set_title('Histogram of Residuals', fontweight='bold')
    axes[0].set_xlabel('Residuals')
    axes[0].set_ylabel('Frequency')
    
    # Q-Q plot
    stats.probplot(residuals, dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Jarque-Bera test
    jb_stat, jb_p = jarque_bera(residuals)
    print(f"   Jarque-Bera Test:")
    print(f"     Statistic: {jb_stat:.4f}")
    print(f"     p-value: {jb_p:.4f}")
    
    if jb_p > 0.05:
        print("   ✓ Residuals appear normally distributed")
    else:
        print("   ⚠ Residuals deviate from normality")
    
    # 3. Homoscedasticity (constant variance)
    print("\n3. HOMOSCEDASTICITY (Constant Variance):")
    
    # Breusch-Pagan test would go here (requires statsmodels)
    print("   Visual check: See residual plot above")
    print("   → Variance should be constant across fitted values")
    
    # 4. Independence (Durbin-Watson)
    print("\n4. INDEPENDENCE OF RESIDUALS:")
    
    # Calculate Durbin-Watson statistic
    dw = np.sum(np.diff(residuals)**2) / np.sum(residuals**2)
    print(f"   Durbin-Watson statistic: {dw:.4f}")
    print("   → Values near 2 indicate no autocorrelation")
    print("   → Values < 2: positive autocorrelation")
    print("   → Values > 2: negative autocorrelation")
    
    # 5. Multicollinearity (VIF)
    print("\n5. MULTICOLLINEARITY CHECK:")
    
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    
    vif_data = pd.DataFrame()
    vif_data["Feature"] = feature_names
    vif_data["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
    
    print(vif_data.round(2))
    print("\n   Interpretation:")
    print("   → VIF = 1: No correlation")
    print("   → VIF < 5: Moderate correlation (acceptable)")
    print("   → VIF > 5: High correlation (problematic)")
    print("   → VIF > 10: Severe multicollinearity")
    
    high_vif = vif_data[vif_data['VIF'] > 5]
    if len(high_vif) > 0:
        print(f"\n   ⚠ Features with high VIF:")
        print(high_vif)
    else:
        print("\n   ✓ No severe multicollinearity detected")
    
    return model, residuals, vif_data

# Example (prepare numeric data first)
X_numeric = df[['tenure', 'MonthlyCharges', 'TotalCharges']].dropna()
y_numeric = df.loc[X_numeric.index, 'Churn'].map({'Yes': 1, 'No': 0})

model, residuals, vif_df = check_regression_assumptions(
    X_numeric.values, 
    y_numeric.values,
    X_numeric.columns.tolist()
)
```

---

## 11. Practical Implementation Guide

### 11.1 Complete Statistical Analysis Workflow

```python
def complete_statistical_analysis_pipeline(df, target_col='Churn'):
    """
    Execute complete statistical analysis for churn dataset.
    """
    print("\n" + "="*80)
    print("COMPLETE STATISTICAL ANALYSIS PIPELINE")
    print("="*80)
    
    results = {}
    
    # PHASE 1: DESCRIPTIVE STATISTICS
    print("\n" + "="*80)
    print("PHASE 1: DESCRIPTIVE STATISTICS")
    print("="*80)
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    for col in numeric_cols:
        print(f"\n{col}:")
        comprehensive_summary(df, col, target_col)
    
    # PHASE 2: DISTRIBUTION ANALYSIS
    print("\n" + "="*80)
    print("PHASE 2: DISTRIBUTION ANALYSIS")
    print("="*80)
    
    for col in numeric_cols:
        test_normality(df[col], col)
    
    # PHASE 3: HYPOTHESIS TESTING
    print("\n" + "="*80)
    print("PHASE 3: HYPOTHESIS TESTING")
    print("="*80)
    
    # T-tests for numeric variables
    for col in numeric_cols:
        churned = df[df# Statistical Approaches for Telco Customer Churn Analysis
## A Comprehensive Dissertation

---

## Table of Contents

1. [Executive Summary](#1-executive-summary)
2. [Introduction to Statistical Analysis in Churn Prediction](#2-introduction)
3. [Descriptive Statistics](#3-descriptive-statistics)
4. [Inferential Statistics](#4-inferential-statistics)
5. [Hypothesis Testing](#5-hypothesis-testing)
6. [Correlation and Association Analysis](#6-correlation-and-association-analysis)
7. [Distribution Analysis](#7-distribution-analysis)
8. [Time Series and Survival Analysis](#8-time-series-and-survival-analysis)
9. [Multivariate Statistical Techniques](#9-multivariate-statistical-techniques)
10. [Statistical Assumptions and Validation](#10-statistical-assumptions)
11. [Practical Implementation Guide](#11-practical-implementation)
12. [Case Studies and Applications](#12-case-studies)
13. [Conclusion](#13-conclusion)

---

## 1. Executive Summary

This dissertation provides a comprehensive guide to statistical approaches essential for analyzing customer churn in telecommunications. We cover 15+ statistical methods, their theoretical foundations, practical applications, and implementation in Python.

### Key Statistical Methods Covered:

- **Descriptive Statistics**: Central tendency, dispersion, distribution shapes
- **Hypothesis Testing**: t-tests, chi-square tests, ANOVA
- **Correlation Analysis**: Pearson, Spearman, point-biserial
- **Distribution Analysis**: Normality tests, Q-Q plots
- **Survival Analysis**: Kaplan-Meier, Cox regression
- **Multivariate Techniques**: PCA, factor analysis, cluster analysis

---

## 2. Introduction to Statistical Analysis in Churn Prediction

### 2.1 Why Statistics Matter in Churn Analysis

Statistical analysis forms the foundation of data-driven churn prediction by:

1. **Quantifying Relationships**: Measure strength between features and churn
2. **Testing Hypotheses**: Validate business assumptions scientifically
3. **Identifying Patterns**: Discover hidden trends in customer behavior
4. **Ensuring Validity**: Verify model assumptions and results
5. **Supporting Decisions**: Provide evidence-based recommendations

### 2.2 The Statistical Analysis Pipeline

```
Data Collection → Descriptive Statistics → Exploratory Analysis →
Hypothesis Testing → Model Building → Validation → Interpretation
```

### 2.3 Types of Variables in Churn Analysis

| Variable Type | Examples | Statistical Methods |
|---------------|----------|---------------------|
| **Binary** | Churn (Yes/No), Gender | Chi-square, logistic regression |
| **Nominal** | Contract type, Payment method | Chi-square, ANOVA |
| **Ordinal** | Satisfaction ratings, Tenure groups | Mann-Whitney U, Kruskal-Wallis |
| **Continuous** | Monthly charges, Tenure (months) | t-tests, correlation, regression |

---

## 3. Descriptive Statistics

Descriptive statistics summarize and describe the main features of your dataset.

### 3.1 Measures of Central Tendency

#### 3.1.1 Mean (Average)

**Definition**: Sum of all values divided by count

**Formula**: 
```
μ = (Σx) / n
```

**When to Use**:
- Continuous variables (tenure, charges)
- Normally distributed data
- No extreme outliers

**Python Implementation**:

```python
import pandas as pd
import numpy as np

# Calculate mean
mean_tenure = df['tenure'].mean()
mean_monthly_charges = df['MonthlyCharges'].mean()

# By churn status
df.groupby('Churn')['tenure'].mean()

# Interpretation
print(f"Average tenure: {mean_tenure:.2f} months")
print(f"Churned customers avg tenure: {df[df['Churn']=='Yes']['tenure'].mean():.2f}")
print(f"Retained customers avg tenure: {df[df['Churn']=='No']['tenure'].mean():.2f}")
```

**Interpretation for Churn**:
- If churned customers have lower mean tenure → New customers at risk
- If churned customers have higher mean charges → Price sensitivity issue

#### 3.1.2 Median

**Definition**: Middle value when data is sorted

**When to Use**:
- Skewed distributions
- Presence of outliers
- Ordinal data

**Python Implementation**:

```python
# Calculate median
median_tenure = df['tenure'].median()

# Compare mean vs median to detect skewness
print(f"Mean tenure: {df['tenure'].mean():.2f}")
print(f"Median tenure: {df['tenure'].median():.2f}")

# If mean > median: Right-skewed (long tail of high values)
# If mean < median: Left-skewed (long tail of low values)
```

#### 3.1.3 Mode

**Definition**: Most frequently occurring value

**When to Use**:
- Categorical variables
- Identify most common category

**Python Implementation**:

```python
# Most common contract type
mode_contract = df['Contract'].mode()[0]
print(f"Most common contract: {mode_contract}")

# Mode by churn status
df[df['Churn']=='Yes']['Contract'].mode()[0]
df[df['Churn']=='No']['Contract'].mode()[0]
```

### 3.2 Measures of Dispersion

#### 3.2.1 Standard Deviation

**Definition**: Average distance from the mean

**Formula**:
```
σ = sqrt(Σ(x - μ)² / n)
```

**Python Implementation**:

```python
# Calculate standard deviation
std_charges = df['MonthlyCharges'].std()

# Coefficient of Variation (CV) - standardized measure
cv = (std_charges / df['MonthlyCharges'].mean()) * 100
print(f"CV: {cv:.2f}% - Shows relative variability")

# Compare variability between groups
churned_std = df[df['Churn']=='Yes']['MonthlyCharges'].std()
retained_std = df[df['Churn']=='No']['MonthlyCharges'].std()

# Higher variability in churned group may indicate pricing issues
```

**Interpretation**:
- Low std dev: Homogeneous customer base
- High std dev: Diverse customer segments
- Compare between churn groups to identify differences

#### 3.2.2 Variance

**Definition**: Square of standard deviation

**Python Implementation**:

```python
variance = df['tenure'].var()

# Variance explained in churn analysis
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_scaled)
explained_variance_ratio = pca.explained_variance_ratio_

print("Variance explained by each component:")
for i, var in enumerate(explained_variance_ratio[:5]):
    print(f"PC{i+1}: {var*100:.2f}%")
```

#### 3.2.3 Range and Interquartile Range (IQR)

**Range**: Maximum - Minimum

**IQR**: Q3 - Q1 (middle 50% of data)

**Python Implementation**:

```python
# Calculate range
data_range = df['MonthlyCharges'].max() - df['MonthlyCharges'].min()

# Calculate IQR
Q1 = df['MonthlyCharges'].quantile(0.25)
Q3 = df['MonthlyCharges'].quantile(0.75)
IQR = Q3 - Q1

# Detect outliers using IQR method
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['MonthlyCharges'] < lower_bound) | 
              (df['MonthlyCharges'] > upper_bound)]

print(f"Number of outliers: {len(outliers)}")
print(f"Outlier percentage: {len(outliers)/len(df)*100:.2f}%")
```

### 3.3 Measures of Shape

#### 3.3.1 Skewness

**Definition**: Measure of asymmetry in distribution

**Interpretation**:
- Skewness = 0: Perfectly symmetric
- Skewness > 0: Right-skewed (tail on right)
- Skewness < 0: Left-skewed (tail on left)

**Python Implementation**:

```python
from scipy.stats import skew, kurtosis

# Calculate skewness
tenure_skew = skew(df['tenure'])
charges_skew = skew(df['MonthlyCharges'])

print(f"Tenure skewness: {tenure_skew:.3f}")
print(f"Monthly charges skewness: {charges_skew:.3f}")

# Interpret
if abs(tenure_skew) < 0.5:
    print("Tenure is approximately symmetric")
elif tenure_skew > 0:
    print("Tenure is right-skewed (many new customers)")
else:
    print("Tenure is left-skewed (many long-term customers)")

# Visualize
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['tenure'], bins=30, edgecolor='black')
axes[0].set_title(f'Tenure Distribution (Skewness: {tenure_skew:.2f})')
axes[0].axvline(df['tenure'].mean(), color='red', linestyle='--', label='Mean')
axes[0].axvline(df['tenure'].median(), color='green', linestyle='--', label='Median')
axes[0].legend()

axes[1].hist(df['MonthlyCharges'], bins=30, edgecolor='black')
axes[1].set_title(f'Monthly Charges (Skewness: {charges_skew:.2f})')
axes[1].axvline(df['MonthlyCharges'].mean(), color='red', linestyle='--', label='Mean')
axes[1].axvline(df['MonthlyCharges'].median(), color='green', linestyle='--', label='Median')
axes[1].legend()

plt.tight_layout()
plt.show()
```

#### 3.3.2 Kurtosis

**Definition**: Measure of "tailedness" or extreme values

**Interpretation**:
- Kurtosis = 3: Normal distribution (mesokurtic)
- Kurtosis > 3: Heavy tails, more outliers (leptokurtic)
- Kurtosis < 3: Light tails, fewer outliers (platykurtic)

**Python Implementation**:

```python
# Calculate excess kurtosis (subtract 3 for comparison to normal)
tenure_kurt = kurtosis(df['tenure'], fisher=True)  # fisher=True gives excess kurtosis

print(f"Tenure excess kurtosis: {tenure_kurt:.3f}")

if tenure_kurt > 0:
    print("→ More extreme values than normal distribution")
    print("→ May need robust statistical methods")
elif tenure_kurt < 0:
    print("→ Fewer extreme values than normal distribution")
    print("→ More uniform distribution")
```

### 3.4 Comprehensive Descriptive Statistics Summary

**Python Implementation**:

```python
def comprehensive_summary(df, column, churn_col='Churn'):
    """
    Generate comprehensive descriptive statistics for a column.
    """
    print(f"\n{'='*60}")
    print(f"COMPREHENSIVE STATISTICS: {column}")
    print(f"{'='*60}\n")
    
    # Overall statistics
    print("Overall Statistics:")
    print(f"  Count: {df[column].count()}")
    print(f"  Mean: {df[column].mean():.2f}")
    print(f"  Median: {df[column].median():.2f}")
    print(f"  Mode: {df[column].mode()[0] if len(df[column].mode()) > 0 else 'N/A'}")
    print(f"  Std Dev: {df[column].std():.2f}")
    print(f"  Variance: {df[column].var():.2f}")
    print(f"  Min: {df[column].min():.2f}")
    print(f"  Max: {df[column].max():.2f}")
    print(f"  Range: {df[column].max() - df[column].min():.2f}")
    
    # Percentiles
    print(f"\nPercentiles:")
    for p in [25, 50, 75, 90, 95, 99]:
        print(f"  {p}th: {df[column].quantile(p/100):.2f}")
    
    # Shape
    print(f"\nDistribution Shape:")
    print(f"  Skewness: {skew(df[column].dropna()):.3f}")
    print(f"  Kurtosis: {kurtosis(df[column].dropna(), fisher=True):.3f}")
    
    # By churn status
    print(f"\nBy Churn Status:")
    for churn_val in df[churn_col].unique():
        subset = df[df[churn_col]==churn_val][column]
        print(f"  {churn_val}:")
        print(f"    Mean: {subset.mean():.2f}")
        print(f"    Median: {subset.median():.2f}")
        print(f"    Std Dev: {subset.std():.2f}")
    
    # Missing values
    missing_pct = (df[column].isnull().sum() / len(df)) * 100
    print(f"\nData Quality:")
    print(f"  Missing: {df[column].isnull().sum()} ({missing_pct:.2f}%)")

# Usage
comprehensive_summary(df, 'tenure')
comprehensive_summary(df, 'MonthlyCharges')
```

---

## 4. Inferential Statistics

Inferential statistics allow us to make predictions and inferences about a population based on sample data.

### 4.1 Confidence Intervals

**Definition**: Range of values that likely contains the true population parameter

**Formula for Mean**:
```
CI = x̄ ± (t * (s / sqrt(n)))
```

Where:
- x̄ = sample mean
- t = t-value from t-distribution
- s = sample standard deviation
- n = sample size

**Python Implementation**:

```python
from scipy import stats

def calculate_confidence_interval(data, confidence=0.95):
    """
    Calculate confidence interval for mean.
    """
    n = len(data)
    mean = np.mean(data)
    std_err = stats.sem(data)  # Standard error of mean
    margin_error = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    
    ci_lower = mean - margin_error
    ci_upper = mean + margin_error
    
    return mean, ci_lower, ci_upper

# Example: Confidence interval for average tenure
churned_tenure = df[df['Churn']=='Yes']['tenure']
retained_tenure = df[df['Churn']=='No']['tenure']

mean_c, lower_c, upper_c = calculate_confidence_interval(churned_tenure)
mean_r, lower_r, upper_r = calculate_confidence_interval(retained_tenure)

print("Average Tenure with 95% Confidence Intervals:")
print(f"Churned: {mean_c:.2f} months [{lower_c:.2f}, {upper_c:.2f}]")
print(f"Retained: {mean_r:.2f} months [{lower_r:.2f}, {upper_r:.2f}]")

# Interpretation
if upper_c < lower_r:
    print("→ Churned customers have significantly lower tenure (no overlap)")
elif lower_c > upper_r:
    print("→ Churned customers have significantly higher tenure")
else:
    print("→ Confidence intervals overlap - difference may not be significant")
```

**Business Application**:
- Estimate true average monthly revenue from customers
- Predict churn rate with confidence bounds
- Compare segments with statistical rigor

### 4.2 Standard Error

**Definition**: Standard deviation of the sampling distribution

**Formula**:
```
SE = σ / sqrt(n)
```

**Python Implementation**:

```python
from scipy.stats import sem

# Calculate standard error for monthly charges
se_charges = sem(df['MonthlyCharges'])

print(f"Standard Error of Monthly Charges: ${se_charges:.2f}")
print(f"This means our sample mean is accurate within ±${se_charges:.2f}")

# Compare standard errors
se_churned = sem(df[df['Churn']=='Yes']['MonthlyCharges'])
se_retained = sem(df[df['Churn']=='No']['MonthlyCharges'])

print(f"\nSE for churned customers: ${se_churned:.2f}")
print(f"SE for retained customers: ${se_retained:.2f}")
```

---

## 5. Hypothesis Testing

Hypothesis testing is crucial for making data-driven decisions about churn drivers.

### 5.1 Framework for Hypothesis Testing

**Standard Process**:

1. **State Hypotheses**:
   - H₀ (Null): No difference/relationship exists
   - H₁ (Alternative): Difference/relationship exists

2. **Choose Significance Level (α)**:
   - Common: α = 0.05 (5% chance of Type I error)

3. **Calculate Test Statistic**

4. **Find p-value**

5. **Make Decision**:
   - If p-value < α: Reject H₀ (significant result)
   - If p-value ≥ α: Fail to reject H₀

### 5.2 Independent Samples t-Test

**Purpose**: Compare means of two independent groups

**Assumptions**:
- Both groups are normally distributed
- Equal variances (or use Welch's t-test)
- Independent observations

**When to Use in Churn Analysis**:
- Compare tenure between churned vs retained
- Compare charges between customer segments

**Python Implementation**:

```python
from scipy.stats import ttest_ind, levene, shapiro

def perform_t_test(group1, group2, group1_name, group2_name, 
                   variable_name, alpha=0.05):
    """
    Perform comprehensive independent t-test with assumption checks.
    """
    print(f"\n{'='*70}")
    print(f"INDEPENDENT T-TEST: {variable_name}")
    print(f"Comparing {group1_name} vs {group2_name}")
    print(f"{'='*70}\n")
    
    # 1. Check normality assumption
    print("1. Normality Tests (Shapiro-Wilk):")
    _, p_norm1 = shapiro(group1.sample(min(5000, len(group1))))  # Sample for large datasets
    _, p_norm2 = shapiro(group2.sample(min(5000, len(group2))))
    
    print(f"   {group1_name}: p-value = {p_norm1:.4f}")
    print(f"   {group2_name}: p-value = {p_norm2:.4f}")
    
    if p_norm1 > 0.05 and p_norm2 > 0.05:
        print("   ✓ Both groups appear normally distributed")
        normality_met = True
    else:
        print("   ⚠ At least one group deviates from normality")
        print("   → Consider using Mann-Whitney U test instead")
        normality_met = False
    
    # 2. Check equal variance assumption
    print("\n2. Equal Variance Test (Levene's Test):")
    _, p_var = levene(group1, group2)
    print(f"   p-value = {p_var:.4f}")
    
    if p_var > 0.05:
        print("   ✓ Variances are equal")
        equal_var = True
    else:
        print("   ⚠ Variances are unequal")
        print("   → Using Welch's t-test (doesn't assume equal variance)")
        equal_var = False
    
    # 3. Perform t-test
    print("\n3. T-Test Results:")
    t_stat, p_value = ttest_ind(group1, group2, equal_var=equal_var)
    
    print(f"   t-statistic: {t_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    print(f"   Significance level: {alpha}")
    
    # 4. Calculate effect size (Cohen's d)
    mean1, mean2 = group1.mean(), group2.mean()
    std1, std2 = group1.std(), group2.std()
    n1, n2 = len(group1), len(group2)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1-1)*std1**2 + (n2-1)*std2**2) / (n1+n2-2))
    cohens_d = (mean1 - mean2) / pooled_std
    
    print(f"\n4. Effect Size (Cohen's d): {cohens_d:.4f}")
    if abs(cohens_d) < 0.2:
        effect = "negligible"
    elif abs(cohens_d) < 0.5:
        effect = "small"
    elif abs(cohens_d) < 0.8:
        effect = "medium"
    else:
        effect = "large"
    print(f"   Effect size is {effect}")
    
    # 5. Interpretation
    print("\n5. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → Reject null hypothesis")
        print(f"   → {group1_name} and {group2_name} have different {variable_name}")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
        print(f"   → Fail to reject null hypothesis")
        print(f"   → Insufficient evidence of difference")
    
    # 6. Descriptive statistics
    print("\n6. Descriptive Statistics:")
    print(f"   {group1_name}: Mean = {mean1:.2f}, SD = {std1:.2f}, n = {n1}")
    print(f"   {group2_name}: Mean = {mean2:.2f}, SD = {std2:.2f}, n = {n2}")
    print(f"   Mean Difference: {abs(mean1 - mean2):.2f}")
    
    return {
        't_statistic': t_stat,
        'p_value': p_value,
        'cohens_d': cohens_d,
        'significant': p_value < alpha
    }

# Example: Compare tenure between churned and retained customers
churned = df[df['Churn']=='Yes']['tenure'].dropna()
retained = df[df['Churn']=='No']['tenure'].dropna()

results = perform_t_test(churned, retained, 
                         'Churned Customers', 'Retained Customers',
                         'Tenure (months)')
```

**Business Interpretation**:

```python
# If significant difference found:
if results['significant']:
    print("\n📊 BUSINESS INSIGHT:")
    print("Churned and retained customers have significantly different tenure.")
    print("→ Action: Focus retention efforts on specific tenure segments")
    print("→ Investigate: What happens at critical tenure milestones?")
```

### 5.3 Paired Samples t-Test

**Purpose**: Compare means of same group at two time points

**When to Use**:
- Before/after retention campaign
- Monthly charges across time periods

**Python Implementation**:

```python
from scipy.stats import ttest_rel

# Example: Compare customer satisfaction before and after intervention
# (hypothetical data)
satisfaction_before = df['satisfaction_before']
satisfaction_after = df['satisfaction_after']

t_stat, p_value = ttest_rel(satisfaction_before, satisfaction_after)

print(f"Paired t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("→ Intervention had significant effect on satisfaction")
```

### 5.4 Chi-Square Test for Independence

**Purpose**: Test relationship between two categorical variables

**When to Use in Churn Analysis**:
- Relationship between Contract type and Churn
- Relationship between Payment method and Churn
- Any categorical variable vs Churn

**Python Implementation**:

```python
from scipy.stats import chi2_contingency

def chi_square_test(df, var1, var2, alpha=0.05):
    """
    Perform chi-square test of independence.
    """
    print(f"\n{'='*70}")
    print(f"CHI-SQUARE TEST: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    # Create contingency table
    contingency_table = pd.crosstab(df[var1], df[var2])
    
    print("1. Contingency Table:")
    print(contingency_table)
    print()
    
    # Perform chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print("2. Test Results:")
    print(f"   Chi-square statistic: {chi2:.4f}")
    print(f"   p-value: {p_value:.4f}")
    print(f"   Degrees of freedom: {dof}")
    
    # Check expected frequencies assumption
    print("\n3. Assumption Check:")
    print("   Expected frequencies (should all be ≥ 5):")
    print(pd.DataFrame(expected, 
                       index=contingency_table.index,
                       columns=contingency_table.columns).round(2))
    
    min_expected = expected.min()
    if min_expected >= 5:
        print(f"   ✓ All expected frequencies ≥ 5 (min: {min_expected:.2f})")
        print("   ✓ Chi-square test is valid")
    else:
        print(f"   ⚠ Some expected frequencies < 5 (min: {min_expected:.2f})")
        print("   ⚠ Consider Fisher's exact test or combine categories")
    
    # Calculate effect size (Cramér's V)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape[0]-1, contingency_table.shape[1]-1)
    cramers_v = np.sqrt(chi2 / (n * min_dim))
    
    print(f"\n4. Effect Size (Cramér's V): {cramers_v:.4f}")
    if cramers_v < 0.1:
        effect = "negligible"
    elif cramers_v < 0.3:
        effect = "small"
    elif cramers_v < 0.5:
        effect = "medium"
    else:
        effect = "large"
    print(f"   Effect size is {effect}")
    
    # Interpretation
    print("\n5. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT ASSOCIATION (p < {alpha})")
        print(f"   → {var1} and {var2} are related")
        print(f"   → Variables are NOT independent")
    else:
        print(f"   ✗ NO SIGNIFICANT ASSOCIATION (p ≥ {alpha})")
        print(f"   → Insufficient evidence of relationship")
    
    # Calculate percentages for interpretation
    print("\n6. Percentage Breakdown:")
    pct_table = pd.crosstab(df[var1], df[var2], normalize='index') * 100
    print(pct_table.round(2))
    
    return {
        'chi2': chi2,
        'p_value': p_value,
        'cramers_v': cramers_v,
        'significant': p_value < alpha,
        'contingency_table': contingency_table
    }

# Example: Test relationship between Contract and Churn
result = chi_square_test(df, 'Contract', 'Churn')

# Business interpretation
if result['significant']:
    print("\n📊 BUSINESS INSIGHT:")
    print("Contract type is significantly related to churn.")
    print("→ Action: Analyze churn rates by contract type")
    print("→ Strategy: Incentivize longer contracts")
```

### 5.5 ANOVA (Analysis of Variance)

**Purpose**: Compare means across 3+ groups

**When to Use**:
- Compare charges across multiple contract types
- Compare tenure across service tiers

**Python Implementation**:

```python
from scipy.stats import f_oneway

def perform_anova(df, group_var, numeric_var, alpha=0.05):
    """
    Perform one-way ANOVA with post-hoc analysis.
    """
    print(f"\n{'='*70}")
    print(f"ONE-WAY ANOVA: {numeric_var} across {group_var}")
    print(f"{'='*70}\n")
    
    # Get groups
    groups = df[group_var].unique()
    group_data = [df[df[group_var]==g][numeric_var].dropna() for g in groups]
    
    # 1. Descriptive statistics
    print("1. Descriptive Statistics by Group:")
    for g, data in zip(groups, group_data):
        print(f"   {g}: Mean={data.mean():.2f}, SD={data.std():.2f}, n={len(data)}")
    
    # 2. Perform ANOVA
    print("\n2. ANOVA Results:")
    f_stat, p_value = f_oneway(*group_data)
    
    print(f"   F-statistic: {f_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # 3. Effect size (eta-squared)
    # Calculate between-group and total sum of squares
    grand_mean = df[numeric_var].mean()
    ss_between = sum([len(data) * (data.mean() - grand_mean)**2 
                      for data in group_data])
    ss_total = sum([(x - grand_mean)**2 for data in group_data for x in data])
    eta_squared = ss_between / ss_total
    
    print(f"\n3. Effect Size (η²): {eta_squared:.4f}")
    print(f"   {eta_squared*100:.2f}% of variance explained by {group_var}")
    
    # 4. Interpretation
    print("\n4. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → At least one group differs significantly")
        print(f"   → Recommend post-hoc tests (Tukey HSD)")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
        print(f"   → All groups have similar means")
    
    # 5. Post-hoc test (Tukey HSD) if significant
    if p_value < alpha:
        from statsmodels.stats.multicomp import pairwise_tukeyhsd
        
        print("\n5. Post-Hoc Analysis (Tukey HSD):")
        tukey = pairwise_tukeyhsd(df[numeric_var], df[group_var], alpha=alpha)
        print(tukey)
    
    return {
        'f_statistic': f_stat,
        'p_value': p_value,
        'eta_squared': eta_squared,
        'significant': p_value < alpha
    }

# Example: Compare monthly charges across contract types
result = perform_anova(df, 'Contract', 'MonthlyCharges')
```

### 5.6 Mann-Whitney U Test (Non-Parametric Alternative)

**Purpose**: Compare distributions of two groups without normality assumption

**When to Use**:
- Data is not normally distributed
- Ordinal data
- Small sample sizes

**Python Implementation**:

```python
from scipy.stats import mannwhitneyu

def mann_whitney_test(group1, group2, group1_name, group2_name, 
                      variable_name, alpha=0.05):
    """
    Perform Mann-Whitney U test (non-parametric alternative to t-test).
    """
    print(f"\n{'='*70}")
    print(f"MANN-WHITNEY U TEST: {variable_name}")
    print(f"Comparing {group1_name} vs {group2_name}")
    print(f"{'='*70}\n")
    
    # Perform test
    u_stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
    
    print("1. Test Results:")
    print(f"   U-statistic: {u_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Calculate effect size (rank-biserial correlation)
    n1, n2 = len(group1), len(group2)
    r = 1 - (2*u_stat) / (n1 * n2)  # rank-biserial correlation
    
    print(f"\n2. Effect Size (rank-biserial r): {r:.4f}")
    
    # Interpretation
    print("\n3. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → Distributions differ significantly")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
    
    # Medians for interpretation
    print("\n4. Median Comparison:")
    print(f"   {group1_name}: Median = {group1.median():.2f}")
    print(f"   {group2_name}: Median = {group2.median():.2f}")
    
    return {
        'u_statistic': u_stat,
        'p_value': p_value,
        'effect_size': r,
        'significant': p_value < alpha
    }

# Example: When data is not normally distributed
churned_charges = df[df['Churn']=='Yes']['MonthlyCharges'].dropna()
retained_charges = df[df['Churn']=='No']['MonthlyCharges'].dropna()

result = mann_whitney_test(churned_charges, retained_charges,
                           'Churned', 'Retained', 'Monthly Charges')
```

---

## 6. Correlation and Association Analysis

Understanding relationships between variables is crucial for feature selection and model building.

### 6.1 Pearson Correlation

**Purpose**: Measure linear relationship between two continuous variables

**Formula**:
```
r = Σ((x - x̄)(y - ȳ)) / sqrt(Σ(x - x̄)² × Σ(y - ȳ)²)
```

**Interpretation**:
- r = 1: Perfect positive correlation
- r = 0: No linear correlation
- r = -1: Perfect negative correlation
- |r| < 0.3: Weak
- 0.3 ≤ |r| < 0.7: Moderate
- |r| ≥ 0.7: Strong

**Python Implementation**:

```python
from scipy.stats import pearsonr

def pearson_correlation_analysis(df, var1, var2, alpha=0.05):
    """
    Comprehensive Pearson correlation analysis.
    """
    print(f"\n{'='*70}")
    print(f"PEARSON CORRELATION: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    # Remove missing values
    data = df[[var1, var2]].dropna()
    
    # Calculate correlation
    r, p_value = pearsonr(data[var1], data[var2])
    
    print("1. Correlation Results:")
    print(f"   Pearson r: {r:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Interpret strength
    if abs(r) < 0.3:
        strength = "weak"
    elif abs(r) < 0.7:
        strength = "moderate"
    else:
        strength = "strong"
    
    direction = "positive" if r > 0 else "negative"
    
    print(f"   Strength: {strength} {direction} correlation")
    
    # Calculate coefficient of determination
    r_squared = r ** 2
    print(f"\n2. Coefficient of Determination (r²): {r_squared:.4f}")
    print(f"   {r_squared*100:.2f}% of variance in {var2} explained by {var1}")
    
    # Statistical significance
    print("\n3. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT CORRELATION (p < {alpha})")
        print(f"   → Relationship is statistically significant")
    else:
        print(f"   ✗ NO SIGNIFICANT CORRELATION (p ≥ {alpha})")
    
    # Visualization
    plt.figure(figsize=(10, 6))
    plt.scatter(data[var1], data[var2], alpha=0.5)
    
    # Add regression line
    z = np.polyfit(data[var1], data[var2], 1)
    p = np.poly1d(z)
    plt.plot(data[var1], p(data[var1]), "r--", linewidth=2, label='Regression line')
    
    plt.xlabel(var1, fontsize=12)
    plt.ylabel(var2, fontsize=12)
    plt.title(f'{var1} vs {var2}\n(r = {r:.3f}, p = {p_value:.4f})', 
              fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {'r': r, 'p_value': p_value, 'r_squared': r_squared}

# Example: Correlation between tenure and total charges
result = pearson_correlation_analysis(df, 'tenure', 'TotalCharges')
```

### 6.2 Spearman Correlation

**Purpose**: Measure monotonic relationship (not necessarily linear)

**When to Use**:
- Ordinal variables
- Non-linear relationships
- Non-normal distributions
- Outliers present

**Python Implementation**:

```python
from scipy.stats import spearmanr

def spearman_correlation_analysis(df, var1, var2, alpha=0.05):
    """
    Spearman rank correlation analysis.
    """
    print(f"\n{'='*70}")
    print(f"SPEARMAN CORRELATION: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    data = df[[var1, var2]].dropna()
    
    # Calculate Spearman correlation
    rho, p_value = spearmanr(data[var1], data[var2])
    
    print("1. Correlation Results:")
    print(f"   Spearman ρ (rho): {rho:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Compare with Pearson
    r_pearson, _ = pearsonr(data[var1], data[var2])
    print(f"\n2. Comparison:")
    print(f"   Pearson r:  {r_pearson:.4f}")
    print(f"   Spearman ρ: {rho:.4f}")
    print(f"   Difference: {abs(r_pearson - rho):.4f}")
    
    if abs(r_pearson - rho) > 0.1:
        print("   ⚠ Large difference suggests non-linear relationship")
    else:
        print("   ✓ Similar values suggest linear relationship")
    
    return {'rho': rho, 'p_value': p_value}

# Example
result = spearman_correlation_analysis(df, 'tenure', 'MonthlyCharges')
```

### 6.3 Point-Biserial Correlation

**Purpose**: Correlation between continuous and binary variable

**When to Use**:
- Relationship between numeric variable and Churn (binary)

**Python Implementation**:

```python
from scipy.stats import pointbiserialr

def point_biserial_analysis(df, continuous_var, binary_var, alpha=0.05):
    """
    Point-biserial correlation for continuous vs binary variable.
    """
    print(f"\n{'='*70}")
    print(f"POINT-BISERIAL CORRELATION")
    print(f"{continuous_var} vs {binary_var}")
    print(f"{'='*70}\n")
    
    # Ensure binary variable is 0/1
    data = df[[continuous_var, binary_var]].dropna()
    if data[binary_var].dtype == 'object':
        binary_map = {data[binary_var].unique()[0]: 0,
                     data[binary_var].unique()[1]: 1}
        data[binary_var] = data[binary_var].map(binary_map)
    
    # Calculate correlation
    r_pb, p_value = pointbiserialr(data[binary_var], data[continuous_var])
    
    print("1. Correlation Results:")
    print(f"   Point-biserial r: {r_pb:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    print(f"\n2. Interpretation:")
    if r_pb > 0:
        print(f"   Positive correlation: Higher {continuous_var} → More likely {binary_var}=1")
    else:
        print(f"   Negative correlation: Higher {continuous_var} → More likely {binary_var}=0")
    
    if p_value < alpha:
        print(f"\n3. Conclusion: SIGNIFICANT relationship (p < {alpha})")
    else:
        print(f"\n3. Conclusion: NO significant relationship (p ≥ {alpha})")
    
    return {'r_pb': r_pb, 'p_value': p_value}

# Example: Tenure vs Churn
result = point_biserial_analysis(df, 'tenure', 'Churn')
```

### 6.4 Correlation Matrix and Heatmap

**Purpose**: Visualize all pairwise correlations

**Python Implementation**:

```python
def comprehensive_correlation_analysis(df, method='pearson'):
    """
    Create comprehensive correlation matrix with visualization.
    """
    # Select numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Calculate correlation matrix
    if method == 'pearson':
        corr_matrix = numeric_df.corr()
    elif method == 'spearman':
        corr_matrix = numeric_df.corr(method='spearman')
    
    # Create visualization
    plt.figure(figsize=(12, 10))
    
    # Create mask for upper triangle
    mask = np.triu(np.ones_like(corr_matrix), k=1)
    
    # Plot heatmap
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', 
                center=0, fmt='.2f', square=True, linewidths=1,
                cbar_kws={"shrink": 0.8})
    
    plt.title(f'{method.capitalize()} Correlation Matrix', 
              fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Find strongest correlations with target (if Churn exists)
    if 'Churn' in corr_matrix.columns:
        print("\nStrongest Correlations with Churn:")
        churn_corr = corr_matrix['Churn'].abs().sort_values(ascending=False)
        print(churn_corr[1:11])  # Top 10, excluding Churn itself
    
    return corr_matrix

# Usage
corr_matrix = comprehensive_correlation_analysis(df, method='pearson')
```

---

## 7. Distribution Analysis

Understanding data distributions is critical for choosing appropriate statistical tests and models.

### 7.1 Normality Tests

#### 7.1.1 Shapiro-Wilk Test

**Purpose**: Test if data comes from normal distribution

**Python Implementation**:

```python
from scipy.stats import shapiro

def test_normality(data, variable_name, alpha=0.05):
    """
    Comprehensive normality testing.
    """
    print(f"\n{'='*70}")
    print(f"NORMALITY TEST: {variable_name}")
    print(f"{'='*70}\n")
    
    # Shapiro-Wilk test
    stat, p_value = shapiro(data.sample(min(5000, len(data))))  # Sample for large datasets
    
    print("1. Shapiro-Wilk Test:")
    print(f"   Statistic: {stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    if p_value > alpha:
        print(f"   ✓ Data appears normally distributed (p > {alpha})")
        normal = True
    else:
        print(f"   ✗ Data deviates from normal distribution (p ≤ {alpha})")
        normal = False
    
    # Visual checks
    

In [ ]:
# Visual checks
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram with normal curve overlay
axes[0].hist(data, bins=30, density=True, alpha=0.7, edgecolor='black')
mu, sigma = data.mean(), data.std()
x = np.linspace(data.min(), data.max(), 100)
axes[0].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal distribution')
axes[0].set_title('Histogram with Normal Curve', fontweight='bold')
axes[0].set_xlabel(variable_name)
axes[0].legend()

# Q-Q plot
stats.probplot(data, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot', fontweight='bold')
axes[1].grid(alpha=0.3)

# Box plot
axes[2].boxplot(data, vert=True)
axes[2].set_title('Box Plot', fontweight='bold')
axes[2].set_ylabel(variable_name)
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n2. Visual Interpretation:")
print("   - Histogram: Should resemble bell curve")
print("   - Q-Q Plot: Points should fall on diagonal line")
print("   - Box Plot: Should be roughly symmetric")

return {'statistic': stat, 'p_value': p_value, 'normal': normal}

# Example
result = test_normality(df['tenure'], 'Tenure (months)')
result = test_normality(df['MonthlyCharges'], 'Monthly Charges ($)')
```

#### 7.1.2 Kolmogorov-Smirnov Test

**Purpose**: Alternative normality test, better for larger samples

**Python Implementation**:

```python
from scipy.stats import kstest

def ks_normality_test(data, variable_name, alpha=0.05):
    """
    Kolmogorov-Smirnov test for normality.
    """
    # Standardize data
    data_std = (data - data.mean()) / data.std()
    
    # Perform KS test
    stat, p_value = kstest(data_std, 'norm')
    
    print(f"\nKolmogorov-Smirnov Test for {variable_name}:")
    print(f"  Statistic: {stat:.4f}")
    print(f"  p-value: {p_value:.4f}")
    
    if p_value > alpha:
        print(f"  ✓ Data appears normally distributed")
    else:
        print(f"  ✗ Data deviates from normality")
    
    return stat, p_value

ks_normality_test(df['tenure'], 'Tenure')
```

### 7.2 Skewness and Kurtosis Tests

**Python Implementation**:

```python
from scipy.stats import skewtest, kurtosistest

def distribution_shape_tests(data, variable_name, alpha=0.05):
    """
    Test skewness and kurtosis significance.
    """
    print(f"\n{'='*60}")
    print(f"DISTRIBUTION SHAPE TESTS: {variable_name}")
    print(f"{'='*60}\n")
    
    # Calculate skewness
    skew_val = skew(data)
    skew_stat, skew_p = skewtest(data)
    
    print("1. Skewness Test:")
    print(f"   Skewness: {skew_val:.4f}")
    print(f"   Test statistic: {skew_stat:.4f}")
    print(f"   p-value: {skew_p:.4f}")
    
    if skew_p < alpha:
        if skew_val > 0:
            print("   ✓ Significantly right-skewed")
        else:
            print("   ✓ Significantly left-skewed")
    else:
        print("   → Skewness not significantly different from 0")
    
    # Calculate kurtosis
    kurt_val = kurtosis(data, fisher=True)
    kurt_stat, kurt_p = kurtosistest(data)
    
    print("\n2. Kurtosis Test:")
    print(f"   Excess kurtosis: {kurt_val:.4f}")
    print(f"   Test statistic: {kurt_stat:.4f}")
    print(f"   p-value: {kurt_p:.4f}")
    
    if kurt_p < alpha:
        if kurt_val > 0:
            print("   ✓ Significantly leptokurtic (heavy tails)")
        else:
            print("   ✓ Significantly platykurtic (light tails)")
    else:
        print("   → Kurtosis not significantly different from normal")

distribution_shape_tests(df['tenure'], 'Tenure')
```

---

## 8. Time Series and Survival Analysis

### 8.1 Survival Analysis (Kaplan-Meier)

**Purpose**: Analyze time until event (churn) occurs

**When to Use**:
- Understand customer lifetime
- Identify critical time periods for churn
- Compare survival across customer segments

**Python Implementation**:

```python
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

def survival_analysis(df, duration_col, event_col, group_col=None):
    """
    Comprehensive survival analysis for churn.
    """
    print(f"\n{'='*70}")
    print("SURVIVAL ANALYSIS (KAPLAN-MEIER)")
    print(f"{'='*70}\n")
    
    # Initialize Kaplan-Meier fitter
    kmf = KaplanMeierFitter()
    
    if group_col is None:
        # Overall survival curve
        kmf.fit(df[duration_col], df[event_col], label='All Customers')
        
        print("Overall Survival Statistics:")
        print(f"  Median survival time: {kmf.median_survival_time_:.2f} months")
        
        # Plot
        plt.figure(figsize=(12, 6))
        kmf.plot_survival_function()
        plt.title('Customer Survival Curve', fontsize=14, fontweight='bold')
        plt.xlabel('Tenure (months)')
        plt.ylabel('Survival Probability')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        
    else:
        # Survival by groups
        print(f"Survival Analysis by {group_col}:\n")
        
        plt.figure(figsize=(12, 6))
        
        groups = df[group_col].unique()
        group_data = []
        
        for group in groups:
            mask = df[group_col] == group
            group_subset = df[mask]
            
            kmf.fit(group_subset[duration_col], 
                   group_subset[event_col],
                   label=str(group))
            
            kmf.plot_survival_function()
            
            print(f"  {group}:")
            print(f"    Median survival: {kmf.median_survival_time_:.2f} months")
            print(f"    1-year survival: {kmf.survival_function_at_times(12).values[0]:.2%}")
            print(f"    2-year survival: {kmf.survival_function_at_times(24).values[0]:.2%}\n")
            
            group_data.append((group_subset[duration_col], group_subset[event_col]))
        
        plt.title(f'Survival Curves by {group_col}', fontsize=14, fontweight='bold')
        plt.xlabel('Tenure (months)')
        plt.ylabel('Survival Probability')
        plt.grid(alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        # Log-rank test if 2 groups
        if len(groups) == 2:
            result = logrank_test(group_data[0][0], group_data[1][0],
                                 group_data[0][1], group_data[1][1])
            
            print(f"Log-Rank Test:")
            print(f"  Test statistic: {result.test_statistic:.4f}")
            print(f"  p-value: {result.p_value:.4f}")
            
            if result.p_value < 0.05:
                print(f"  ✓ Survival curves are significantly different")
            else:
                print(f"  ✗ No significant difference between groups")

# Prepare data (tenure as duration, Churn as event)
df_survival = df.copy()
df_survival['Churn_binary'] = (df_survival['Churn'] == 'Yes').astype(int)

# Overall survival
survival_analysis(df_survival, 'tenure', 'Churn_binary')

# Survival by contract type
survival_analysis(df_survival, 'tenure', 'Churn_binary', 'Contract')
```

**Business Interpretation**:

```python
print("\n📊 BUSINESS INSIGHTS FROM SURVIVAL ANALYSIS:")
print("1. Median survival time tells us typical customer lifetime")
print("2. Steep drops indicate critical churn periods")
print("3. Compare curves across segments to prioritize interventions")
print("4. 1-year survival rate = retention rate at 12 months")
```

### 8.2 Cox Proportional Hazards Model

**Purpose**: Identify factors affecting time to churn

**Python Implementation**:

```python
from lifelines import CoxPHFitter

def cox_regression_analysis(df, duration_col, event_col, covariates):
    """
    Cox proportional hazards model for churn.
    """
    print(f"\n{'='*70}")
    print("COX PROPORTIONAL HAZARDS MODEL")
    print(f"{'='*70}\n")
    
    # Prepare data
    analysis_df = df[[duration_col, event_col] + covariates].dropna()
    
    # Fit model
    cph = CoxPHFitter()
    cph.fit(analysis_df, duration_col=duration_col, event_col=event_col)
    
    # Display results
    print("Model Summary:")
    print(cph.summary)
    
    print("\n\nInterpretation of Hazard Ratios:")
    print("(exp(coef) = Hazard Ratio)")
    print("-" * 60)
    
    for var in covariates:
        coef = cph.params_[var]
        hr = np.exp(coef)
        p_val = cph.summary.loc[var, 'p']
        
        print(f"\n{var}:")
        print(f"  Hazard Ratio: {hr:.4f}")
        
        if hr > 1:
            print(f"  → Increases churn risk by {(hr-1)*100:.1f}%")
        else:
            print(f"  → Decreases churn risk by {(1-hr)*100:.1f}%")
        
        if p_val < 0.05:
            print(f"  ✓ Statistically significant (p={p_val:.4f})")
        else:
            print(f"  ✗ Not significant (p={p_val:.4f})")
    
    # Plot
    cph.plot()
    plt.title('Hazard Ratios with 95% CI', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return cph

# Example with numeric predictors
covariates = ['MonthlyCharges', 'SeniorCitizen', 'Partner', 'Dependents']
cph_model = cox_regression_analysis(df_survival, 'tenure', 'Churn_binary', covariates)
```

---

## 9. Multivariate Statistical Techniques

### 9.1 Principal Component Analysis (PCA)

**Purpose**: Reduce dimensionality while preserving variance

**When to Use**:
- Many correlated features
- Visualization of high-dimensional data
- Feature extraction

**Python Implementation**:

```python
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def comprehensive_pca_analysis(df, n_components=None):
    """
    Complete PCA analysis with interpretation.
    """
    print(f"\n{'='*70}")
    print("PRINCIPAL COMPONENT ANALYSIS (PCA)")
    print(f"{'='*70}\n")
    
    # Select numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[numeric_cols].dropna()
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Fit PCA
    if n_components is None:
        n_components = min(len(numeric_cols), len(X))
    
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # 1. Variance explained
    print("1. Variance Explained:")
    cumsum_var = np.cumsum(pca.explained_variance_ratio_)
    
    for i, (var, cumvar) in enumerate(zip(pca.explained_variance_ratio_, cumsum_var)):
        print(f"   PC{i+1}: {var*100:.2f}% (Cumulative: {cumvar*100:.2f}%)")
        if cumvar >= 0.95:
            print(f"   → 95% variance explained with {i+1} components")
            break
    
    # 2. Scree plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Variance explained
    axes[0].bar(range(1, len(pca.explained_variance_ratio_)+1), 
                pca.explained_variance_ratio_, alpha=0.7)
    axes[0].plot(range(1, len(cumsum_var)+1), cumsum_var, 'r-o', linewidth=2)
    axes[0].set_xlabel('Principal Component', fontweight='bold')
    axes[0].set_ylabel('Variance Explained', fontweight='bold')
    axes[0].set_title('Scree Plot', fontweight='bold')
    axes[0].axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Biplot (first 2 components)
    axes[1].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
    axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontweight='bold')
    axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontweight='bold')
    axes[1].set_title('PCA Biplot (First 2 Components)', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 3. Component loadings
    print("\n2. Top Feature Loadings for First 3 Components:")
    loadings = pd.DataFrame(
        pca.components_.T,
        columns=[f'PC{i+1}' for i in range(pca.n_components_)],
        index=numeric_cols
    )
    
    for i in range(min(3, pca.n_components_)):
        print(f"\n   PC{i+1} - Top Features:")
        top_features = loadings[f'PC{i+1}'].abs().sort_values(ascending=False).head(5)
        for feature, loading in top_features.items():
            actual_loading = loadings.loc[feature, f'PC{i+1}']
            print(f"     {feature}: {actual_loading:.3f}")
    
    return pca, X_pca, loadings

# Example
pca_model, X_transformed, loadings = comprehensive_pca_analysis(df)
```

### 9.2 Factor Analysis

**Purpose**: Identify latent factors underlying observed variables

**Python Implementation**:

```python
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

def factor_analysis_comprehensive(df, n_factors=3):
    """
    Comprehensive factor analysis.
    """
    print(f"\n{'='*70}")
    print("FACTOR ANALYSIS")
    print(f"{'='*70}\n")
    
    # Select numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[numeric_cols].dropna()
    
    # 1. Test suitability for factor analysis
    print("1. Suitability Tests:")
    
    # KMO Test
    kmo_all, kmo_model = calculate_kmo(X)
    print(f"   Kaiser-Meyer-Olkin (KMO) Test: {kmo_model:.3f}")
    if kmo_model >= 0.6:
        print("   ✓ Data suitable for factor analysis (KMO > 0.6)")
    else:
        print("   ⚠ Data may not be suitable (KMO < 0.6)")
    
    # Bartlett's Test
    chi_square, p_value = calculate_bartlett_sphericity(X)
    print(f"\n   Bartlett's Test of Sphericity:")
    print(f"     Chi-square: {chi_square:.2f}")
    print(f"     p-value: {p_value:.4f}")
    if p_value < 0.05:
        print("   ✓ Variables are correlated (suitable for FA)")
    else:
        print("   ⚠ Variables may not be sufficiently correlated")
    
    # 2. Fit factor analysis
    print(f"\n2. Fitting {n_factors}-Factor Model:")
    
    fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax')
    fa.fit(X)
    
    # Get factor loadings
    loadings = pd.DataFrame(
        fa.loadings_,
        index=numeric_cols,
        columns=[f'Factor{i+1}' for i in range(n_factors)]
    )
    
    print("\n   Factor Loadings:")
    print(loadings.round(3))
    
    # 3. Variance explained
    variance = fa.get_factor_variance()
    
    print("\n3. Variance Explained:")
    print(f"   Proportional variance: {variance[1]}")
    print(f"   Cumulative variance: {variance[2]}")
    
    # 4. Interpret factors
    print("\n4. Factor Interpretation:")
    for i in range(n_factors):
        print(f"\n   Factor {i+1} - Top Loaded Variables:")
        top_vars = loadings[f'Factor{i+1}'].abs().sort_values(ascending=False).head(5)
        for var, loading in top_vars.items():
            actual = loadings.loc[var, f'Factor{i+1}']
            print(f"     {var}: {actual:.3f}")
    
    return fa, loadings

# Example
fa_model, factor_loadings = factor_analysis_comprehensive(df, n_factors=3)
```

### 9.3 Cluster Analysis

**Purpose**: Group similar customers together

**Python Implementation**:

```python
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

def cluster_analysis(df, n_clusters_range=range(2, 11)):
    """
    K-means clustering with optimal cluster selection.
    """
    print(f"\n{'='*70}")
    print("CLUSTER ANALYSIS (K-MEANS)")
    print(f"{'='*70}\n")
    
    # Prepare data
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[numeric_cols].dropna()
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 1. Determine optimal number of clusters
    print("1. Finding Optimal Number of Clusters:")
    
    inertias = []
    silhouette_scores = []
    
    for k in n_clusters_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    
    # Plot elbow curve and silhouette scores
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Elbow method
    axes[0].plot(n_clusters_range, inertias, 'bo-', linewidth=2)
    axes[0].set_xlabel('Number of Clusters', fontweight='bold')
    axes[0].set_ylabel('Inertia', fontweight='bold')
    axes[0].set_title('Elbow Method', fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Silhouette score
    axes[1].plot(n_clusters_range, silhouette_scores, 'ro-', linewidth=2)
    axes[1].set_xlabel('Number of Clusters', fontweight='bold')
    axes[1].set_ylabel('Silhouette Score', fontweight='bold')
    axes[1].set_title('Silhouette Analysis', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Select optimal k (highest silhouette score)
    optimal_k = n_clusters_range[np.argmax(silhouette_scores)]
    print(f"\n   Optimal number of clusters: {optimal_k}")
    print(f"   Best silhouette score: {max(silhouette_scores):.3f}")
    
    # 2. Fit final model
    print(f"\n2. Fitting {optimal_k}-Cluster Model:")
    
    kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    clusters = kmeans_final.fit_predict(X_scaled)
    
    # Add clusters to dataframe
    df_clustered = X.copy()
    df_clustered['Cluster'] = clusters
    
    # 3. Analyze clusters
    print(f"\n3. Cluster Profiles:")
    
    for i in range(optimal_k):
        cluster_data = df_clustered[df_clustered['Cluster'] == i]
        print(f"\n   Cluster {i} (n={len(cluster_data)}):")
        print(f"     Mean values:")
        for col in numeric_cols[:5]:  # Show top 5 features
            print(f"       {col}: {cluster_data[col].mean():.2f}")
    
    # 4. Visualize clusters (2D PCA)
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.6)
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontweight='bold')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontweight='bold')
    plt.title('Customer Segments (K-Means Clustering)', fontweight='bold', fontsize=14)
    plt.colorbar(scatter, label='Cluster')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return kmeans_final, clusters, df_clustered

# Example
kmeans_model, cluster_labels, df_with_clusters = cluster_analysis(df)
```

---

## 10. Statistical Assumptions and Validation

### 10.1 Checking Linear Regression Assumptions

**Python Implementation**:

```python
from sklearn.linear_model import LinearRegression
from scipy.stats import jarque_bera

def check_regression_assumptions(X, y, feature_names):
    """
    Comprehensive check of linear regression assumptions.
    """
    print(f"\n{'='*70}")
    print("LINEAR REGRESSION ASSUMPTIONS CHECK")
    print(f"{'='*70}\n")
    
    # Fit model
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    residuals = y - y_pred
    
    # 1. Linearity
    print("1. LINEARITY (Residuals vs Fitted Values):")
    plt.figure(figsize=(10, 6))
    plt.scatter(y_pred, residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
    plt.xlabel('Fitted Values', fontweight='bold')
    plt.ylabel('Residuals', fontweight='bold')
    plt.title('Residual Plot', fontweight='bold')
    plt.grid(alpha=0.3)
    plt.show()
    print("   → Pattern should be random scatter around zero")
    print("   → Funnel shape indicates heteroscedasticity")
    print("   → Curve indicates non-linearity")
    
    # 2. Normality of residuals
    print("\n2. NORMALITY OF RESIDUALS:")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histogram
    axes[0].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
    axes[0].set_title('Histogram of Residuals', fontweight='bold')
    axes[0].set_xlabel('Residuals')
    axes[0].set_ylabel('Frequency')
    
    # Q-Q plot
    stats.probplot(residuals, dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Jarque-Bera test
    jb_stat, jb_p = jarque_bera(residuals)
    print(f"   Jarque-Bera Test:")
    print(f"     Statistic: {jb_stat:.4f}")
    print(f"     p-value: {jb_p:.4f}")
    
    if jb_p > 0.05:
        print("   ✓ Residuals appear normally distributed")
    else:
        print("   ⚠ Residuals deviate from normality")
    
    # 3. Homoscedasticity (constant variance)
    print("\n3. HOMOSCEDASTICITY (Constant Variance):")
    
    # Breusch-Pagan test would go here (requires statsmodels)
    print("   Visual check: See residual plot above")
    print("   → Variance should be constant across fitted values")
    
    # 4. Independence (Durbin-Watson)
    print("\n4. INDEPENDENCE OF RESIDUALS:")
    
    # Calculate Durbin-Watson statistic
    dw = np.sum(np.diff(residuals)**2) / np.sum(residuals**2)
    print(f"   Durbin-Watson statistic: {dw:.4f}")
    print("   → Values near 2 indicate no autocorrelation")
    print("   → Values < 2: positive autocorrelation")
    print("   → Values > 2: negative autocorrelation")
    
    # 5. Multicollinearity (VIF)
    print("\n5. MULTICOLLINEARITY CHECK:")
    
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    
    vif_data = pd.DataFrame()
    vif_data["Feature"] = feature_names
    vif_data["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
    
    print(vif_data.round(2))
    print("\n   Interpretation:")
    print("   → VIF = 1: No correlation")
    print("   → VIF < 5: Moderate correlation (acceptable)")
    print("   → VIF > 5: High correlation (problematic)")
    print("   → VIF > 10: Severe multicollinearity")
    
    high_vif = vif_data[vif_data['VIF'] > 5]
    if len(high_vif) > 0:
        print(f"\n   ⚠ Features with high VIF:")
        print(high_vif)
    else:
        print("\n   ✓ No severe multicollinearity detected")
    
    return model, residuals, vif_data

# Example (prepare numeric data first)
X_numeric = df[['tenure', 'MonthlyCharges', 'TotalCharges']].dropna()
y_numeric = df.loc[X_numeric.index, 'Churn'].map({'Yes': 1, 'No': 0})

model, residuals, vif_df = check_regression_assumptions(
    X_numeric.values, 
    y_numeric.values,
    X_numeric.columns.tolist()
)
```

---

## 11. Practical Implementation Guide

### 11.1 Complete Statistical Analysis Workflow

```python
def complete_statistical_analysis_pipeline(df, target_col='Churn'):
    """
    Execute complete statistical analysis for churn dataset.
    """
    print("\n" + "="*80)
    print("COMPLETE STATISTICAL ANALYSIS PIPELINE")
    print("="*80)
    
    results = {}
    
    # PHASE 1: DESCRIPTIVE STATISTICS
    print("\n" + "="*80)
    print("PHASE 1: DESCRIPTIVE STATISTICS")
    print("="*80)
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    for col in numeric_cols:
        print(f"\n{col}:")
        comprehensive_summary(df, col, target_col)
    
    # PHASE 2: DISTRIBUTION ANALYSIS
    print("\n" + "="*80)
    print("PHASE 2: DISTRIBUTION ANALYSIS")
    print("="*80)
    
    for col in numeric_cols:
        test_normality(df[col], col)
    
    # PHASE 3: HYPOTHESIS TESTING
    print("\n" + "="*80)
    print("PHASE 3: HYPOTHESIS TESTING")
    print("="*80)
    
    # T-tests for numeric variables
    for col in numeric_cols:
        churned = df[df[target_col]=='Yes'][col].dropna()
        retained = df[df[target_col]=='No'][col].dropna()
        
        results[f'{col}_ttest'] = perform_t_test(
            churned, retained, 'Churned', 'Retained', col
        )
    
    # Chi-square tests for categorical variables
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    categorical_cols = [c for c in categorical_cols if c != target_col]
    
    for col in categorical_cols:
        results[f'{col}_chisquare'] = chi_square_test(df, col, target_col)
    
    # PHASE 4: CORRELATION ANALYSIS
    print("\n" + "="*80)
    print("PHASE 4: CORRELATION ANALYSIS")
    print("="*80)
    
    results['correlation_matrix'] = comprehensive_correlation_analysis(df, method='pearson')
    
    # PHASE 5: MULTIVARIATE ANALYSIS
    print("\n" + "="*80)
    print("PHASE 5: MULTIVARIATE ANALYSIS")
    print("="*80)
    
    # PCA
    pca_model, X_pca, loadings = comprehensive_pca_analysis(df)
    results['pca'] = {'model': pca_model, 'transformed': X_pca, 'loadings': loadings}
    
    # Clustering
    kmeans_model, clusters, df_clustered = cluster_analysis(df)
    results['clusters'] = {'model': kmeans_model, 'labels': clusters, 'data': df_clustered}
    
    # PHASE 6: SURVIVAL ANALYSIS
    print("\n" + "="*80)
    print("PHASE 6: SURVIVAL ANALYSIS")
    print("="*80)
    
    if 'tenure' in df.columns:
        df_survival = df.copy()
        df_survival['Churn_binary'] = (df_survival[target_col] == 'Yes').astype(int)
        survival_analysis(df_survival, 'tenure', 'Churn_binary')
    
    # FINAL SUMMARY
    print("\n" + "="*80)
    print("ANALYSIS COMPLETE - KEY FINDINGS SUMMARY")
    print("="*80)
    
    print("\n1. Significant Differences (t-tests):")
    for key, result in results.items():
        if '_ttest' in key and result.get('significant'):
            print(f"   ✓ {key.replace('_ttest', '')}: p={result['p_value']:.4f}")
    
    print("\n2. Significant Associations (chi-square):")
    for key, result in results.items():
        if '_chisquare' in key and result.get('significant'):
            print(f"   ✓ {key.replace('_chisquare', '')}: p={result['p_value']:.4f}")
    
    return results

# Execute complete pipeline
results = complete_statistical_analysis_pipeline(df, target_col='Churn')
```

### 11.2 Statistical Report Generator

```python
def generate_statistical_report(df, output_file='statistical_report.txt'):
    """
    Generate comprehensive statistical report.
    """
    import sys
    from datetime import datetime
    
    # Redirect output to file
    original_stdout = sys.stdout
    
    with open(output_file, 'w') as f:
        sys.stdout = f
        
        print("="*80)
        print("TELCO CUSTOMER CHURN - STATISTICAL ANALYSIS REPORT")
        print("="*80)
        print(f"\nGenerated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Dataset: {len(df)} customers, {len(df.columns)} features")
        
        # Execute analyses
        results = complete_statistical_analysis_pipeline(df)
        
        print("\n" + "="*80)
        print("END OF REPORT")
        print("="*80)
    
    # Restore stdout
    sys.stdout = original_stdout
    
    print(f"\n✓ Statistical report saved to: {output_file}")
    return results

# Generate report
report_results = generate_statistical_report(df)
```

---

## 12. Case Studies and Applications

### 12.1 Case Study: Identifying High-Risk Customers

**Objective**: Use statistical methods to segment customers by churn risk

**Approach**:

```python
def identify_high_risk_customers(df):
    """
    Statistical approach to identify high-risk customers.
    """
    print("\n" + "="*70)
    print("CASE STUDY: IDENTIFYING HIGH-RISK CUSTOMERS")
    print("="*70)
    
    # Step 1: Statistical profiling
    print("\n1. STATISTICAL PROFILING OF CHURNED CUSTOMERS:")
    
    churned = df[df['Churn'] == 'Yes']
    retained = df[df['Churn'] == 'No']
    
    # Identify significant differences
    risk_factors = {}
    
    # Numeric variables
    numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
    
    for col in numeric_cols:
        result = perform_t_test(
            churned[col].dropna(),
            retained[col].dropna(),
            'Churned', 'Retained', col
        )
        
        if result['significant']:
            risk_factors[col] = {
                'type': 'numeric',
                'churned_mean': churned[col].mean(),
                'retained_mean': retained[col].mean(),
                'effect_size': result['cohens_d']
            }
    
    # Categorical variables
    categorical_cols = ['Contract', 'PaymentMethod', 'InternetService']
    
    for col in categorical_cols:
        result = chi_square_test(df, col, 'Churn')
        
        if result['significant']:
            # Calculate churn rates by category
            churn_rates = df.groupby(col)['Churn'].apply(
                lambda x: (x == 'Yes').sum() / len(x) * 100
            )
            
            risk_factors[col] = {
                'type': 'categorical',
                'churn_rates': churn_rates.to_dict(),
                'highest_risk': churn_rates.idxmax()
            }
    
    # Step 2: Create risk score
    print("\n2. CREATING RISK SCORE:")
    
    df_risk = df.copy()
    df_risk['risk_score'] = 0
    
    # Add points based on statistical findings
    for factor, info in risk_factors.items():
        if info['type'] == 'numeric':
            if info['churned_mean'] < info['retained_mean']:
                # Lower values indicate higher risk (e.g., tenure)
                threshold = info['retained_mean']
                df_risk['risk_score'] += (df_risk[factor] < threshold).astype(int)
            else:
                # Higher values indicate higher risk (e.g., charges)
                threshold = info['retained_mean']
                df_risk['risk_score'] += (df_risk[factor] > threshold).astype(int)
        
        elif info['type'] == 'categorical':
            # Highest risk category gets a point
            highest_risk_cat = info['highest_risk']
            df_risk['risk_score'] += (df_risk[factor] == highest_risk_cat).astype(int)
    
    # Normalize to 0-100 scale
    max_score = df_risk['risk_score'].max()
    df_risk['risk_score_normalized'] = (df_risk['risk_score'] / max_score * 100).round(0)
    
    # Step 3: Validate risk score
    print("\n3. VALIDATING RISK SCORE:")
    
    # Correlation with actual churn
    df_risk['Churn_binary'] = (df_risk['Churn'] == 'Yes').astype(int)
    correlation = df_risk[['risk_score_normalized', 'Churn_binary']].corr().iloc[0, 1]
    
    print(f"   Correlation with actual churn: {correlation:.4f}")
    
    # ROC-AUC
    from sklearn.metrics import roc_auc_score, roc_curve
    
    roc_auc = roc_auc_score(df_risk['Churn_binary'], df_risk['risk_score_normalized'])
    print(f"   ROC-AUC: {roc_auc:.4f}")
    
    # Plot ROC curve
    fpr, tpr, thresholds = roc_curve(df_risk['Churn_binary'], df_risk['risk_score_normalized'])
    
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, linewidth=2, label=f'Risk Score (AUC={roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate', fontweight='bold')
    plt.ylabel('True Positive Rate', fontweight='bold')
    plt.title('ROC Curve - Risk Score Validation', fontweight='bold', fontsize=14)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Step 4: Segment customers
    print("\n4. CUSTOMER SEGMENTATION BY RISK:")
    
    # Create risk categories
    df_risk['risk_category'] = pd.cut(
        df_risk['risk_score_normalized'],
        bins=[0, 33, 66, 100],
        labels=['Low Risk', 'Medium Risk', 'High Risk']
    )
    
    # Analyze by segment
    for category in ['Low Risk', 'Medium Risk', 'High Risk']:
        segment = df_risk[df_risk['risk_category'] == category]
        actual_churn_rate = (segment['Churn'] == 'Yes').sum() / len(segment) * 100
        
        print(f"\n   {category}:")
        print(f"     Customers: {len(segment):,}")
        print(f"     Actual churn rate: {actual_churn_rate:.2f}%")
        print(f"     Avg risk score: {segment['risk_score_normalized'].mean():.1f}")
    
    return df_risk, risk_factors

# Execute case study
df_with_risk, risk_factors = identify_high_risk_customers(df)
```

### 12.2 Case Study: A/B Testing Retention Campaign

**Objective**: Test if retention campaign reduces churn

**Approach**:

```python
def ab_test_retention_campaign(df_control, df_treatment, metric='Churn'):
    """
    Statistical A/B test for retention campaign.
    """
    print("\n" + "="*70)
    print("CASE STUDY: A/B TESTING RETENTION CAMPAIGN")
    print("="*70)
    
    # Convert churn to binary
    control_churn = (df_control[metric] == 'Yes').astype(int)
    treatment_churn = (df_treatment[metric] == 'Yes').astype(int)
    
    # Step 1: Sample size and power analysis
    print("\n1. SAMPLE SIZE ANALYSIS:")
    print(f"   Control group: {len(df_control):,} customers")
    print(f"   Treatment group: {len(df_treatment):,} customers")
    
    # Step 2: Check for balance
    print("\n2. BALANCE CHECK (Pre-treatment characteristics):")
    
    numeric_vars = ['tenure', 'MonthlyCharges']
    
    for var in numeric_vars:
        t_stat, p_value = ttest_ind(df_control[var], df_treatment[var])
        print(f"   {var}: p={p_value:.4f}", end="")
        if p_value > 0.05:
            print(" ✓ Balanced")
        else:
            print(" ⚠ Imbalanced - adjust analysis")
    
    # Step 3: Test for difference in churn rates
    print("\n3. HYPOTHESIS TEST:")
    print("   H₀: Treatment has no effect on churn")
    print("   H₁: Treatment reduces churn")
    
    # Two-proportion z-test
    from statsmodels.stats.proportion import proportions_ztest
    
    count = np.array([control_churn.sum(), treatment_churn.sum()])
    nobs = np.array([len(control_churn), len(treatment_churn)])
    
    z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')
    
    print(f"\n   Control churn rate: {control_churn.mean()*100:.2f}%")
    print(f"   Treatment churn rate: {treatment_churn.mean()*100:.2f}%")
    print(f"   Difference: {(control_churn.mean() - treatment_churn.mean())*100:.2f} percentage points")
    print(f"\n   Z-statistic: {z_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("\n   ✓ SIGNIFICANT EFFECT: Treatment reduces churn")
    else:
        print("\n   ✗ NO SIGNIFICANT EFFECT: Treatment does not significantly reduce churn")
    
    # Step 4: Effect size and confidence interval
    print("\n4. EFFECT SIZE & CONFIDENCE INTERVAL:")
    
    # Relative risk
    rr = (treatment_churn.mean() / control_churn.mean())
    print(f"   Relative Risk: {rr:.4f}")
    print(f"   → Treatment group has {(1-rr)*100:.1f}% lower churn risk")
    
    # Absolute risk reduction
    arr = (control_churn.mean() - treatment_churn.mean()) * 100
    print(f"   Absolute Risk Reduction: {arr:.2f} percentage points")
    
    # Number needed to treat
    if arr > 0:
        nnt = 100 / arr
        print(f"   Number Needed to Treat: {nnt:.1f}")
        print(f"   → Need to treat {nnt:.0f} customers to prevent 1 churn")
    
    # Step 5: Business impact
    print("\n5. BUSINESS IMPACT:")
    
    customers_saved = len(df_treatment) * (control_churn.mean() - treatment_churn.mean())
    cost_per_customer = 50  # Assumed campaign cost
    total_cost = len(df_treatment) * cost_per_customer
    value_per_customer = 500  # Assumed customer lifetime value
    total_value = customers_saved * value_per_customer
    roi = (total_value - total_cost) / total_cost * 100
    
    print(f"   Customers saved: {customers_saved:.0f}")
    print(f"   Campaign cost: ${total_cost:,.0f}")
    print(f"   Value generated: ${total_value:,.0f}")
    print(f"   ROI: {roi:.1f}%")
    
    return {
        'p_value': p_value,
        'significant': p_value < 0.05,
        'effect_size': arr,
        'roi': roi
    }

# Example (create simulated treatment data)
# In practice, this would be actual A/B test data
np.random.seed(42)
control_indices = df.sample(frac=0.5).index
treatment_indices = df.drop(control_indices).index

df_control = df.loc[control_indices]
df_treatment = df.loc[treatment_indices].copy()

# Simulate treatment effect (reduce churn by 5%)
treatment_effect = np.random.random(len(df_treatment)) < 0.05
df_treatment.loc[df_treatment['Churn']=='Yes', 'Churn'] = np.where(
    treatment_effect[df_treatment['Churn']=='Yes'],
    'No',
    df_treatment.loc[df_treatment['Churn']=='Yes', 'Churn']
)

results = ab_test_retention_campaign(df_control, df_treatment)
```

---

## 13. Conclusion

### 13.1 Summary of Statistical Methods

This dissertation has covered comprehensive statistical approaches for telco churn analysis:

**Descriptive Statistics**:
- Central tendency (mean, median, mode)
- Dispersion (std dev, variance, IQR)
- Shape (skewness, kurtosis)

**Inferential Statistics**:
- Confidence intervals
- Hypothesis testing (t-tests, chi-square, ANOVA)
- Effect size calculations

**Correlation Analysis**:
- Pearson, Spearman, point-biserial correlations
- Correlation matrices and heatmaps

**Advanced Techniques**:
- Survival analysis (Kaplan-Meier, Cox regression)
- Multivariate methods (PCA, factor analysis, clustering)
- A/B testing and experimental design

### 13.2 Best Practices

1. **Always Check Assumptions**: Don't blindly apply tests
2. **Report Effect Sizes**: p-values alone aren't enough
3. **Use Multiple Methods**: Triangulate findings
4. **Visualize Results**: Graphics aid interpretation
5. **Consider Business Context**: Statistical significance ≠ practical significance
6. **Document Thoroughly**: Reproducibility matters

### 13.3 Common Pitfalls to Avoid

```python
print("""
COMMON STATISTICAL PITFALLS IN CHURN ANALYSIS:

1. P-HACKING
   ❌ Testing many hypotheses until finding p<0.05
   ✓ Pre-register hypotheses, adjust for multiple testing

2. IGNORING ASSUMPTIONS
   ❌ Using t-test on non-normal data
   ✓ Check assumptions, use appropriate alternatives

3. CONFUSING CORRELATION WITH CAUSATION
   ❌ "High charges cause churn"
   ✓ "High charges are associated with churn"

4. CHERRY-PICKING DATA
   ❌ Removing "outliers" to get desired results
   ✓ Document all data cleaning decisions

5. SMALL SAMPLE SIZES
   ❌ Drawing conclusions from n=30
   ✓ Conduct power analysis, collect sufficient data

6. IGNORING EFFECT SIZE
   ❌ "p<0.001, must be important!"
   ✓ Check practical significance (Cohen's d, etc.)

7. MULTIPLE TESTING WITHOUT CORRECTION
   ❌ Running 100 tests at α=0.05
   ✓ Use Bonferroni or FDR correction

8. TREATING NON-INDEPENDENT DATA AS INDEPENDENT
   ❌ Repeated measures from same customers
   ✓ Use appropriate paired/repeated measures tests
""")
```

### 13.4 Recommended Statistical Workflow

```python
def recommended_statistical_workflow():
    """
    Step-by-step guide for statistical analysis.
    """
    workflow = """
    RECOMMENDED STATISTICAL ANALYSIS WORKFLOW:
    
    STEP 1: EXPLORATION
    □ Load and inspect data
    □ Calculate descriptive statistics
    □ Create visualizations
    □ Identify patterns and anomalies
    
    STEP 2: ASSUMPTION CHECKING
    □ Test normality (Shapiro-Wilk, Q-Q plots)
    □ Check for outliers
    □ Assess homogeneity of variance
    □ Verify independence
    
    STEP 3: HYPOTHESIS FORMULATION
    □ Define null and alternative hypotheses
    □ Set significance level (α = 0.05)
    □ Determine required sample size
    
    STEP 4: TEST SELECTION
    □ Choose appropriate statistical test
    □ Consider parametric vs non-parametric
    □ Account for multiple comparisons
    
    STEP 5: EXECUTION
    □ Perform statistical tests
    □ Calculate effect sizes
    □ Compute confidence intervals
    
    STEP 6: INTERPRETATION
    □ Assess statistical significance
    □ Evaluate practical significance
    □ Consider business context
    
    STEP 7: VALIDATION
    □ Check assumptions post-hoc
    □ Perform sensitivity analyses
    □ Validate on holdout set
    
    STEP 8: REPORTING
    □ Document all decisions
    □ Create visualizations
    □ Write clear interpretations
    □ Include limitations
    """
    
    print(workflow)
    return workflow

workflow = recommended_statistical_workflow()
```

### 13.5 Final Recommendations

**For Practitioners**:

1. **Start Simple**: Begin with descriptive statistics and visualizations
2. **Build Up**: Progress to hypothesis testing and multivariate methods
3. **Validate Everything**: Check assumptions and validate results
4. **Think Business**: Always connect statistics to business outcomes
5. **Stay Updated**: Statistical methods evolve - keep learning

**For Stakeholders**:

1. Statistical significance ≠ business importance
2. Confidence intervals provide more information than p-values
3. Effect sizes tell you "how much" not just "if"
4. Correlation doesn't imply causation
5. All models are wrong, but some are useful

### 13.6 Resources for Further Learning

```python
resources = {
    'Books': [
        'Statistics in Plain English by Timothy Urdan',
        'Practical Statistics for Data Scientists by Bruce & Bruce',
        'The Elements of Statistical Learning by Hastie, Tibshirani, Friedman'
    ],
    'Online Courses': [
        'Khan Academy - Statistics and Probability',
        'Coursera - Statistical Inference',
        'DataCamp - Statistical Thinking in Python'
    ],
    'Python Libraries': [
        'scipy.stats - Statistical functions',
        'statsmodels - Statistical models',
        'pingouin - Statistical tests',
        'lifelines - Survival analysis'
    ],
    'Documentation': [
        'SciPy Stats Documentation',
        'Statsmodels Documentation',
        'Scikit-learn User Guide'
    ]
}

print("\n📚 RECOMMENDED RESOURCES:\n")
for category, items in resources.items():
    print(f"{category}:")
    for item in items:
        print(f"  • {item}")
    print()
```

---

## Appendix: Quick Reference Guide

```python
def statistical_methods_quick_reference():
    """
    Quick reference for choosing statistical methods.
    """
    guide = """
    STATISTICAL METHODS QUICK REFERENCE
    ═══════════════════════════════════
    
    COMPARING TWO GROUPS:
    ────────────────────
    Continuous data:
      • Normal distribution → Independent t-test
      • Non-normal → Mann-Whitney U test
      • Paired samples → Paired t-test
    
    Categorical data:
      • 2x2 table → Chi-square or Fisher's exact
      • Larger tables → Chi-square test
    
    COMPARING 3+ GROUPS:
    ───────────────────
    Continuous data:
      • Normal → One-way ANOVA
      • Non-normal → Kruskal-Wallis test
    
    RELATIONSHIPS:
    ─────────────
    Two continuous variables:
      • Linear relationship → Pearson correlation
      • Monotonic → Spearman correlation
    
    Continuous + Binary:
      • Point-biserial correlation
    
    Two categorical:
      • Chi-square test
    
    PREDICTION:
    ──────────
    Binary outcome:
      • Logistic regression
      • Survival analysis (if time involved)
    
    Continuous outcome:
      • Linear regression
      • Multiple regression
    
    DIMENSION REDUCTION:
    ──────────────────
    • PCA - uncorrelated components
    • Factor analysis - latent factors
    
    GROUPING:
    ────────
    • K-means clustering
    • Hierarchical clustering
    """
    
    print(guide)
    return guide

quick_ref = statistical_methods_quick_reference()
```

---

**END OF DISSERTATION**

This comprehensive guide provides the statistical foundation necessary for rigorous telco churn analysis. Apply these methods thoughtfully, always considering both statistical and business significance.

**Remember**: Statistics is a tool for understanding, not just for achieving p<0.05!# Statistical Approaches for Telco Customer Churn Analysis
## A Comprehensive Dissertation

---

## Table of Contents

1. [Executive Summary](#1-executive-summary)
2. [Introduction to Statistical Analysis in Churn Prediction](#2-introduction)
3. [Descriptive Statistics](#3-descriptive-statistics)
4. [Inferential Statistics](#4-inferential-statistics)
5. [Hypothesis Testing](#5-hypothesis-testing)
6. [Correlation and Association Analysis](#6-correlation-and-association-analysis)
7. [Distribution Analysis](#7-distribution-analysis)
8. [Time Series and Survival Analysis](#8-time-series-and-survival-analysis)
9. [Multivariate Statistical Techniques](#9-multivariate-statistical-techniques)
10. [Statistical Assumptions and Validation](#10-statistical-assumptions)
11. [Practical Implementation Guide](#11-practical-implementation)
12. [Case Studies and Applications](#12-case-studies)
13. [Conclusion](#13-conclusion)

---

## 1. Executive Summary

This dissertation provides a comprehensive guide to statistical approaches essential for analyzing customer churn in telecommunications. We cover 15+ statistical methods, their theoretical foundations, practical applications, and implementation in Python.

### Key Statistical Methods Covered:

- **Descriptive Statistics**: Central tendency, dispersion, distribution shapes
- **Hypothesis Testing**: t-tests, chi-square tests, ANOVA
- **Correlation Analysis**: Pearson, Spearman, point-biserial
- **Distribution Analysis**: Normality tests, Q-Q plots
- **Survival Analysis**: Kaplan-Meier, Cox regression
- **Multivariate Techniques**: PCA, factor analysis, cluster analysis

---

## 2. Introduction to Statistical Analysis in Churn Prediction

### 2.1 Why Statistics Matter in Churn Analysis

Statistical analysis forms the foundation of data-driven churn prediction by:

1. **Quantifying Relationships**: Measure strength between features and churn
2. **Testing Hypotheses**: Validate business assumptions scientifically
3. **Identifying Patterns**: Discover hidden trends in customer behavior
4. **Ensuring Validity**: Verify model assumptions and results
5. **Supporting Decisions**: Provide evidence-based recommendations

### 2.2 The Statistical Analysis Pipeline

```
Data Collection → Descriptive Statistics → Exploratory Analysis →
Hypothesis Testing → Model Building → Validation → Interpretation
```

### 2.3 Types of Variables in Churn Analysis

| Variable Type | Examples | Statistical Methods |
|---------------|----------|---------------------|
| **Binary** | Churn (Yes/No), Gender | Chi-square, logistic regression |
| **Nominal** | Contract type, Payment method | Chi-square, ANOVA |
| **Ordinal** | Satisfaction ratings, Tenure groups | Mann-Whitney U, Kruskal-Wallis |
| **Continuous** | Monthly charges, Tenure (months) | t-tests, correlation, regression |

---

## 3. Descriptive Statistics

Descriptive statistics summarize and describe the main features of your dataset.

### 3.1 Measures of Central Tendency

#### 3.1.1 Mean (Average)

**Definition**: Sum of all values divided by count

**Formula**: 
```
μ = (Σx) / n
```

**When to Use**:
- Continuous variables (tenure, charges)
- Normally distributed data
- No extreme outliers

**Python Implementation**:

```python
import pandas as pd
import numpy as np

# Calculate mean
mean_tenure = df['tenure'].mean()
mean_monthly_charges = df['MonthlyCharges'].mean()

# By churn status
df.groupby('Churn')['tenure'].mean()

# Interpretation
print(f"Average tenure: {mean_tenure:.2f} months")
print(f"Churned customers avg tenure: {df[df['Churn']=='Yes']['tenure'].mean():.2f}")
print(f"Retained customers avg tenure: {df[df['Churn']=='No']['tenure'].mean():.2f}")
```

**Interpretation for Churn**:
- If churned customers have lower mean tenure → New customers at risk
- If churned customers have higher mean charges → Price sensitivity issue

#### 3.1.2 Median

**Definition**: Middle value when data is sorted

**When to Use**:
- Skewed distributions
- Presence of outliers
- Ordinal data

**Python Implementation**:

```python
# Calculate median
median_tenure = df['tenure'].median()

# Compare mean vs median to detect skewness
print(f"Mean tenure: {df['tenure'].mean():.2f}")
print(f"Median tenure: {df['tenure'].median():.2f}")

# If mean > median: Right-skewed (long tail of high values)
# If mean < median: Left-skewed (long tail of low values)
```

#### 3.1.3 Mode

**Definition**: Most frequently occurring value

**When to Use**:
- Categorical variables
- Identify most common category

**Python Implementation**:

```python
# Most common contract type
mode_contract = df['Contract'].mode()[0]
print(f"Most common contract: {mode_contract}")

# Mode by churn status
df[df['Churn']=='Yes']['Contract'].mode()[0]
df[df['Churn']=='No']['Contract'].mode()[0]
```

### 3.2 Measures of Dispersion

#### 3.2.1 Standard Deviation

**Definition**: Average distance from the mean

**Formula**:
```
σ = sqrt(Σ(x - μ)² / n)
```

**Python Implementation**:

```python
# Calculate standard deviation
std_charges = df['MonthlyCharges'].std()

# Coefficient of Variation (CV) - standardized measure
cv = (std_charges / df['MonthlyCharges'].mean()) * 100
print(f"CV: {cv:.2f}% - Shows relative variability")

# Compare variability between groups
churned_std = df[df['Churn']=='Yes']['MonthlyCharges'].std()
retained_std = df[df['Churn']=='No']['MonthlyCharges'].std()

# Higher variability in churned group may indicate pricing issues
```

**Interpretation**:
- Low std dev: Homogeneous customer base
- High std dev: Diverse customer segments
- Compare between churn groups to identify differences

#### 3.2.2 Variance

**Definition**: Square of standard deviation

**Python Implementation**:

```python
variance = df['tenure'].var()

# Variance explained in churn analysis
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_scaled)
explained_variance_ratio = pca.explained_variance_ratio_

print("Variance explained by each component:")
for i, var in enumerate(explained_variance_ratio[:5]):
    print(f"PC{i+1}: {var*100:.2f}%")
```

#### 3.2.3 Range and Interquartile Range (IQR)

**Range**: Maximum - Minimum

**IQR**: Q3 - Q1 (middle 50% of data)

**Python Implementation**:

```python
# Calculate range
data_range = df['MonthlyCharges'].max() - df['MonthlyCharges'].min()

# Calculate IQR
Q1 = df['MonthlyCharges'].quantile(0.25)
Q3 = df['MonthlyCharges'].quantile(0.75)
IQR = Q3 - Q1

# Detect outliers using IQR method
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['MonthlyCharges'] < lower_bound) | 
              (df['MonthlyCharges'] > upper_bound)]

print(f"Number of outliers: {len(outliers)}")
print(f"Outlier percentage: {len(outliers)/len(df)*100:.2f}%")
```

### 3.3 Measures of Shape

#### 3.3.1 Skewness

**Definition**: Measure of asymmetry in distribution

**Interpretation**:
- Skewness = 0: Perfectly symmetric
- Skewness > 0: Right-skewed (tail on right)
- Skewness < 0: Left-skewed (tail on left)

**Python Implementation**:

```python
from scipy.stats import skew, kurtosis

# Calculate skewness
tenure_skew = skew(df['tenure'])
charges_skew = skew(df['MonthlyCharges'])

print(f"Tenure skewness: {tenure_skew:.3f}")
print(f"Monthly charges skewness: {charges_skew:.3f}")

# Interpret
if abs(tenure_skew) < 0.5:
    print("Tenure is approximately symmetric")
elif tenure_skew > 0:
    print("Tenure is right-skewed (many new customers)")
else:
    print("Tenure is left-skewed (many long-term customers)")

# Visualize
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['tenure'], bins=30, edgecolor='black')
axes[0].set_title(f'Tenure Distribution (Skewness: {tenure_skew:.2f})')
axes[0].axvline(df['tenure'].mean(), color='red', linestyle='--', label='Mean')
axes[0].axvline(df['tenure'].median(), color='green', linestyle='--', label='Median')
axes[0].legend()

axes[1].hist(df['MonthlyCharges'], bins=30, edgecolor='black')
axes[1].set_title(f'Monthly Charges (Skewness: {charges_skew:.2f})')
axes[1].axvline(df['MonthlyCharges'].mean(), color='red', linestyle='--', label='Mean')
axes[1].axvline(df['MonthlyCharges'].median(), color='green', linestyle='--', label='Median')
axes[1].legend()

plt.tight_layout()
plt.show()
```

#### 3.3.2 Kurtosis

**Definition**: Measure of "tailedness" or extreme values

**Interpretation**:
- Kurtosis = 3: Normal distribution (mesokurtic)
- Kurtosis > 3: Heavy tails, more outliers (leptokurtic)
- Kurtosis < 3: Light tails, fewer outliers (platykurtic)

**Python Implementation**:

```python
# Calculate excess kurtosis (subtract 3 for comparison to normal)
tenure_kurt = kurtosis(df['tenure'], fisher=True)  # fisher=True gives excess kurtosis

print(f"Tenure excess kurtosis: {tenure_kurt:.3f}")

if tenure_kurt > 0:
    print("→ More extreme values than normal distribution")
    print("→ May need robust statistical methods")
elif tenure_kurt < 0:
    print("→ Fewer extreme values than normal distribution")
    print("→ More uniform distribution")
```

### 3.4 Comprehensive Descriptive Statistics Summary

**Python Implementation**:

```python
def comprehensive_summary(df, column, churn_col='Churn'):
    """
    Generate comprehensive descriptive statistics for a column.
    """
    print(f"\n{'='*60}")
    print(f"COMPREHENSIVE STATISTICS: {column}")
    print(f"{'='*60}\n")
    
    # Overall statistics
    print("Overall Statistics:")
    print(f"  Count: {df[column].count()}")
    print(f"  Mean: {df[column].mean():.2f}")
    print(f"  Median: {df[column].median():.2f}")
    print(f"  Mode: {df[column].mode()[0] if len(df[column].mode()) > 0 else 'N/A'}")
    print(f"  Std Dev: {df[column].std():.2f}")
    print(f"  Variance: {df[column].var():.2f}")
    print(f"  Min: {df[column].min():.2f}")
    print(f"  Max: {df[column].max():.2f}")
    print(f"  Range: {df[column].max() - df[column].min():.2f}")
    
    # Percentiles
    print(f"\nPercentiles:")
    for p in [25, 50, 75, 90, 95, 99]:
        print(f"  {p}th: {df[column].quantile(p/100):.2f}")
    
    # Shape
    print(f"\nDistribution Shape:")
    print(f"  Skewness: {skew(df[column].dropna()):.3f}")
    print(f"  Kurtosis: {kurtosis(df[column].dropna(), fisher=True):.3f}")
    
    # By churn status
    print(f"\nBy Churn Status:")
    for churn_val in df[churn_col].unique():
        subset = df[df[churn_col]==churn_val][column]
        print(f"  {churn_val}:")
        print(f"    Mean: {subset.mean():.2f}")
        print(f"    Median: {subset.median():.2f}")
        print(f"    Std Dev: {subset.std():.2f}")
    
    # Missing values
    missing_pct = (df[column].isnull().sum() / len(df)) * 100
    print(f"\nData Quality:")
    print(f"  Missing: {df[column].isnull().sum()} ({missing_pct:.2f}%)")

# Usage
comprehensive_summary(df, 'tenure')
comprehensive_summary(df, 'MonthlyCharges')
```

---

## 4. Inferential Statistics

Inferential statistics allow us to make predictions and inferences about a population based on sample data.

### 4.1 Confidence Intervals

**Definition**: Range of values that likely contains the true population parameter

**Formula for Mean**:
```
CI = x̄ ± (t * (s / sqrt(n)))
```

Where:
- x̄ = sample mean
- t = t-value from t-distribution
- s = sample standard deviation
- n = sample size

**Python Implementation**:

```python
from scipy import stats

def calculate_confidence_interval(data, confidence=0.95):
    """
    Calculate confidence interval for mean.
    """
    n = len(data)
    mean = np.mean(data)
    std_err = stats.sem(data)  # Standard error of mean
    margin_error = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    
    ci_lower = mean - margin_error
    ci_upper = mean + margin_error
    
    return mean, ci_lower, ci_upper

# Example: Confidence interval for average tenure
churned_tenure = df[df['Churn']=='Yes']['tenure']
retained_tenure = df[df['Churn']=='No']['tenure']

mean_c, lower_c, upper_c = calculate_confidence_interval(churned_tenure)
mean_r, lower_r, upper_r = calculate_confidence_interval(retained_tenure)

print("Average Tenure with 95% Confidence Intervals:")
print(f"Churned: {mean_c:.2f} months [{lower_c:.2f}, {upper_c:.2f}]")
print(f"Retained: {mean_r:.2f} months [{lower_r:.2f}, {upper_r:.2f}]")

# Interpretation
if upper_c < lower_r:
    print("→ Churned customers have significantly lower tenure (no overlap)")
elif lower_c > upper_r:
    print("→ Churned customers have significantly higher tenure")
else:
    print("→ Confidence intervals overlap - difference may not be significant")
```

**Business Application**:
- Estimate true average monthly revenue from customers
- Predict churn rate with confidence bounds
- Compare segments with statistical rigor

### 4.2 Standard Error

**Definition**: Standard deviation of the sampling distribution

**Formula**:
```
SE = σ / sqrt(n)
```

**Python Implementation**:

```python
from scipy.stats import sem

# Calculate standard error for monthly charges
se_charges = sem(df['MonthlyCharges'])

print(f"Standard Error of Monthly Charges: ${se_charges:.2f}")
print(f"This means our sample mean is accurate within ±${se_charges:.2f}")

# Compare standard errors
se_churned = sem(df[df['Churn']=='Yes']['MonthlyCharges'])
se_retained = sem(df[df['Churn']=='No']['MonthlyCharges'])

print(f"\nSE for churned customers: ${se_churned:.2f}")
print(f"SE for retained customers: ${se_retained:.2f}")
```

---

## 5. Hypothesis Testing

Hypothesis testing is crucial for making data-driven decisions about churn drivers.

### 5.1 Framework for Hypothesis Testing

**Standard Process**:

1. **State Hypotheses**:
   - H₀ (Null): No difference/relationship exists
   - H₁ (Alternative): Difference/relationship exists

2. **Choose Significance Level (α)**:
   - Common: α = 0.05 (5% chance of Type I error)

3. **Calculate Test Statistic**

4. **Find p-value**

5. **Make Decision**:
   - If p-value < α: Reject H₀ (significant result)
   - If p-value ≥ α: Fail to reject H₀

### 5.2 Independent Samples t-Test

**Purpose**: Compare means of two independent groups

**Assumptions**:
- Both groups are normally distributed
- Equal variances (or use Welch's t-test)
- Independent observations

**When to Use in Churn Analysis**:
- Compare tenure between churned vs retained
- Compare charges between customer segments

**Python Implementation**:

```python
from scipy.stats import ttest_ind, levene, shapiro

def perform_t_test(group1, group2, group1_name, group2_name, 
                   variable_name, alpha=0.05):
    """
    Perform comprehensive independent t-test with assumption checks.
    """
    print(f"\n{'='*70}")
    print(f"INDEPENDENT T-TEST: {variable_name}")
    print(f"Comparing {group1_name} vs {group2_name}")
    print(f"{'='*70}\n")
    
    # 1. Check normality assumption
    print("1. Normality Tests (Shapiro-Wilk):")
    _, p_norm1 = shapiro(group1.sample(min(5000, len(group1))))  # Sample for large datasets
    _, p_norm2 = shapiro(group2.sample(min(5000, len(group2))))
    
    print(f"   {group1_name}: p-value = {p_norm1:.4f}")
    print(f"   {group2_name}: p-value = {p_norm2:.4f}")
    
    if p_norm1 > 0.05 and p_norm2 > 0.05:
        print("   ✓ Both groups appear normally distributed")
        normality_met = True
    else:
        print("   ⚠ At least one group deviates from normality")
        print("   → Consider using Mann-Whitney U test instead")
        normality_met = False
    
    # 2. Check equal variance assumption
    print("\n2. Equal Variance Test (Levene's Test):")
    _, p_var = levene(group1, group2)
    print(f"   p-value = {p_var:.4f}")
    
    if p_var > 0.05:
        print("   ✓ Variances are equal")
        equal_var = True
    else:
        print("   ⚠ Variances are unequal")
        print("   → Using Welch's t-test (doesn't assume equal variance)")
        equal_var = False
    
    # 3. Perform t-test
    print("\n3. T-Test Results:")
    t_stat, p_value = ttest_ind(group1, group2, equal_var=equal_var)
    
    print(f"   t-statistic: {t_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    print(f"   Significance level: {alpha}")
    
    # 4. Calculate effect size (Cohen's d)
    mean1, mean2 = group1.mean(), group2.mean()
    std1, std2 = group1.std(), group2.std()
    n1, n2 = len(group1), len(group2)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1-1)*std1**2 + (n2-1)*std2**2) / (n1+n2-2))
    cohens_d = (mean1 - mean2) / pooled_std
    
    print(f"\n4. Effect Size (Cohen's d): {cohens_d:.4f}")
    if abs(cohens_d) < 0.2:
        effect = "negligible"
    elif abs(cohens_d) < 0.5:
        effect = "small"
    elif abs(cohens_d) < 0.8:
        effect = "medium"
    else:
        effect = "large"
    print(f"   Effect size is {effect}")
    
    # 5. Interpretation
    print("\n5. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → Reject null hypothesis")
        print(f"   → {group1_name} and {group2_name} have different {variable_name}")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
        print(f"   → Fail to reject null hypothesis")
        print(f"   → Insufficient evidence of difference")
    
    # 6. Descriptive statistics
    print("\n6. Descriptive Statistics:")
    print(f"   {group1_name}: Mean = {mean1:.2f}, SD = {std1:.2f}, n = {n1}")
    print(f"   {group2_name}: Mean = {mean2:.2f}, SD = {std2:.2f}, n = {n2}")
    print(f"   Mean Difference: {abs(mean1 - mean2):.2f}")
    
    return {
        't_statistic': t_stat,
        'p_value': p_value,
        'cohens_d': cohens_d,
        'significant': p_value < alpha
    }

# Example: Compare tenure between churned and retained customers
churned = df[df['Churn']=='Yes']['tenure'].dropna()
retained = df[df['Churn']=='No']['tenure'].dropna()

results = perform_t_test(churned, retained, 
                         'Churned Customers', 'Retained Customers',
                         'Tenure (months)')
```

**Business Interpretation**:

```python
# If significant difference found:
if results['significant']:
    print("\n📊 BUSINESS INSIGHT:")
    print("Churned and retained customers have significantly different tenure.")
    print("→ Action: Focus retention efforts on specific tenure segments")
    print("→ Investigate: What happens at critical tenure milestones?")
```

### 5.3 Paired Samples t-Test

**Purpose**: Compare means of same group at two time points

**When to Use**:
- Before/after retention campaign
- Monthly charges across time periods

**Python Implementation**:

```python
from scipy.stats import ttest_rel

# Example: Compare customer satisfaction before and after intervention
# (hypothetical data)
satisfaction_before = df['satisfaction_before']
satisfaction_after = df['satisfaction_after']

t_stat, p_value = ttest_rel(satisfaction_before, satisfaction_after)

print(f"Paired t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("→ Intervention had significant effect on satisfaction")
```

### 5.4 Chi-Square Test for Independence

**Purpose**: Test relationship between two categorical variables

**When to Use in Churn Analysis**:
- Relationship between Contract type and Churn
- Relationship between Payment method and Churn
- Any categorical variable vs Churn

**Python Implementation**:

```python
from scipy.stats import chi2_contingency

def chi_square_test(df, var1, var2, alpha=0.05):
    """
    Perform chi-square test of independence.
    """
    print(f"\n{'='*70}")
    print(f"CHI-SQUARE TEST: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    # Create contingency table
    contingency_table = pd.crosstab(df[var1], df[var2])
    
    print("1. Contingency Table:")
    print(contingency_table)
    print()
    
    # Perform chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print("2. Test Results:")
    print(f"   Chi-square statistic: {chi2:.4f}")
    print(f"   p-value: {p_value:.4f}")
    print(f"   Degrees of freedom: {dof}")
    
    # Check expected frequencies assumption
    print("\n3. Assumption Check:")
    print("   Expected frequencies (should all be ≥ 5):")
    print(pd.DataFrame(expected, 
                       index=contingency_table.index,
                       columns=contingency_table.columns).round(2))
    
    min_expected = expected.min()
    if min_expected >= 5:
        print(f"   ✓ All expected frequencies ≥ 5 (min: {min_expected:.2f})")
        print("   ✓ Chi-square test is valid")
    else:
        print(f"   ⚠ Some expected frequencies < 5 (min: {min_expected:.2f})")
        print("   ⚠ Consider Fisher's exact test or combine categories")
    
    # Calculate effect size (Cramér's V)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape[0]-1, contingency_table.shape[1]-1)
    cramers_v = np.sqrt(chi2 / (n * min_dim))
    
    print(f"\n4. Effect Size (Cramér's V): {cramers_v:.4f}")
    if cramers_v < 0.1:
        effect = "negligible"
    elif cramers_v < 0.3:
        effect = "small"
    elif cramers_v < 0.5:
        effect = "medium"
    else:
        effect = "large"
    print(f"   Effect size is {effect}")
    
    # Interpretation
    print("\n5. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT ASSOCIATION (p < {alpha})")
        print(f"   → {var1} and {var2} are related")
        print(f"   → Variables are NOT independent")
    else:
        print(f"   ✗ NO SIGNIFICANT ASSOCIATION (p ≥ {alpha})")
        print(f"   → Insufficient evidence of relationship")
    
    # Calculate percentages for interpretation
    print("\n6. Percentage Breakdown:")
    pct_table = pd.crosstab(df[var1], df[var2], normalize='index') * 100
    print(pct_table.round(2))
    
    return {
        'chi2': chi2,
        'p_value': p_value,
        'cramers_v': cramers_v,
        'significant': p_value < alpha,
        'contingency_table': contingency_table
    }

# Example: Test relationship between Contract and Churn
result = chi_square_test(df, 'Contract', 'Churn')

# Business interpretation
if result['significant']:
    print("\n📊 BUSINESS INSIGHT:")
    print("Contract type is significantly related to churn.")
    print("→ Action: Analyze churn rates by contract type")
    print("→ Strategy: Incentivize longer contracts")
```

### 5.5 ANOVA (Analysis of Variance)

**Purpose**: Compare means across 3+ groups

**When to Use**:
- Compare charges across multiple contract types
- Compare tenure across service tiers

**Python Implementation**:

```python
from scipy.stats import f_oneway

def perform_anova(df, group_var, numeric_var, alpha=0.05):
    """
    Perform one-way ANOVA with post-hoc analysis.
    """
    print(f"\n{'='*70}")
    print(f"ONE-WAY ANOVA: {numeric_var} across {group_var}")
    print(f"{'='*70}\n")
    
    # Get groups
    groups = df[group_var].unique()
    group_data = [df[df[group_var]==g][numeric_var].dropna() for g in groups]
    
    # 1. Descriptive statistics
    print("1. Descriptive Statistics by Group:")
    for g, data in zip(groups, group_data):
        print(f"   {g}: Mean={data.mean():.2f}, SD={data.std():.2f}, n={len(data)}")
    
    # 2. Perform ANOVA
    print("\n2. ANOVA Results:")
    f_stat, p_value = f_oneway(*group_data)
    
    print(f"   F-statistic: {f_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # 3. Effect size (eta-squared)
    # Calculate between-group and total sum of squares
    grand_mean = df[numeric_var].mean()
    ss_between = sum([len(data) * (data.mean() - grand_mean)**2 
                      for data in group_data])
    ss_total = sum([(x - grand_mean)**2 for data in group_data for x in data])
    eta_squared = ss_between / ss_total
    
    print(f"\n3. Effect Size (η²): {eta_squared:.4f}")
    print(f"   {eta_squared*100:.2f}% of variance explained by {group_var}")
    
    # 4. Interpretation
    print("\n4. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → At least one group differs significantly")
        print(f"   → Recommend post-hoc tests (Tukey HSD)")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
        print(f"   → All groups have similar means")
    
    # 5. Post-hoc test (Tukey HSD) if significant
    if p_value < alpha:
        from statsmodels.stats.multicomp import pairwise_tukeyhsd
        
        print("\n5. Post-Hoc Analysis (Tukey HSD):")
        tukey = pairwise_tukeyhsd(df[numeric_var], df[group_var], alpha=alpha)
        print(tukey)
    
    return {
        'f_statistic': f_stat,
        'p_value': p_value,
        'eta_squared': eta_squared,
        'significant': p_value < alpha
    }

# Example: Compare monthly charges across contract types
result = perform_anova(df, 'Contract', 'MonthlyCharges')
```

### 5.6 Mann-Whitney U Test (Non-Parametric Alternative)

**Purpose**: Compare distributions of two groups without normality assumption

**When to Use**:
- Data is not normally distributed
- Ordinal data
- Small sample sizes

**Python Implementation**:

```python
from scipy.stats import mannwhitneyu

def mann_whitney_test(group1, group2, group1_name, group2_name, 
                      variable_name, alpha=0.05):
    """
    Perform Mann-Whitney U test (non-parametric alternative to t-test).
    """
    print(f"\n{'='*70}")
    print(f"MANN-WHITNEY U TEST: {variable_name}")
    print(f"Comparing {group1_name} vs {group2_name}")
    print(f"{'='*70}\n")
    
    # Perform test
    u_stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
    
    print("1. Test Results:")
    print(f"   U-statistic: {u_stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Calculate effect size (rank-biserial correlation)
    n1, n2 = len(group1), len(group2)
    r = 1 - (2*u_stat) / (n1 * n2)  # rank-biserial correlation
    
    print(f"\n2. Effect Size (rank-biserial r): {r:.4f}")
    
    # Interpretation
    print("\n3. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT DIFFERENCE (p < {alpha})")
        print(f"   → Distributions differ significantly")
    else:
        print(f"   ✗ NO SIGNIFICANT DIFFERENCE (p ≥ {alpha})")
    
    # Medians for interpretation
    print("\n4. Median Comparison:")
    print(f"   {group1_name}: Median = {group1.median():.2f}")
    print(f"   {group2_name}: Median = {group2.median():.2f}")
    
    return {
        'u_statistic': u_stat,
        'p_value': p_value,
        'effect_size': r,
        'significant': p_value < alpha
    }

# Example: When data is not normally distributed
churned_charges = df[df['Churn']=='Yes']['MonthlyCharges'].dropna()
retained_charges = df[df['Churn']=='No']['MonthlyCharges'].dropna()

result = mann_whitney_test(churned_charges, retained_charges,
                           'Churned', 'Retained', 'Monthly Charges')
```

---

## 6. Correlation and Association Analysis

Understanding relationships between variables is crucial for feature selection and model building.

### 6.1 Pearson Correlation

**Purpose**: Measure linear relationship between two continuous variables

**Formula**:
```
r = Σ((x - x̄)(y - ȳ)) / sqrt(Σ(x - x̄)² × Σ(y - ȳ)²)
```

**Interpretation**:
- r = 1: Perfect positive correlation
- r = 0: No linear correlation
- r = -1: Perfect negative correlation
- |r| < 0.3: Weak
- 0.3 ≤ |r| < 0.7: Moderate
- |r| ≥ 0.7: Strong

**Python Implementation**:

```python
from scipy.stats import pearsonr

def pearson_correlation_analysis(df, var1, var2, alpha=0.05):
    """
    Comprehensive Pearson correlation analysis.
    """
    print(f"\n{'='*70}")
    print(f"PEARSON CORRELATION: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    # Remove missing values
    data = df[[var1, var2]].dropna()
    
    # Calculate correlation
    r, p_value = pearsonr(data[var1], data[var2])
    
    print("1. Correlation Results:")
    print(f"   Pearson r: {r:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Interpret strength
    if abs(r) < 0.3:
        strength = "weak"
    elif abs(r) < 0.7:
        strength = "moderate"
    else:
        strength = "strong"
    
    direction = "positive" if r > 0 else "negative"
    
    print(f"   Strength: {strength} {direction} correlation")
    
    # Calculate coefficient of determination
    r_squared = r ** 2
    print(f"\n2. Coefficient of Determination (r²): {r_squared:.4f}")
    print(f"   {r_squared*100:.2f}% of variance in {var2} explained by {var1}")
    
    # Statistical significance
    print("\n3. Statistical Conclusion:")
    if p_value < alpha:
        print(f"   ✓ SIGNIFICANT CORRELATION (p < {alpha})")
        print(f"   → Relationship is statistically significant")
    else:
        print(f"   ✗ NO SIGNIFICANT CORRELATION (p ≥ {alpha})")
    
    # Visualization
    plt.figure(figsize=(10, 6))
    plt.scatter(data[var1], data[var2], alpha=0.5)
    
    # Add regression line
    z = np.polyfit(data[var1], data[var2], 1)
    p = np.poly1d(z)
    plt.plot(data[var1], p(data[var1]), "r--", linewidth=2, label='Regression line')
    
    plt.xlabel(var1, fontsize=12)
    plt.ylabel(var2, fontsize=12)
    plt.title(f'{var1} vs {var2}\n(r = {r:.3f}, p = {p_value:.4f})', 
              fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {'r': r, 'p_value': p_value, 'r_squared': r_squared}

# Example: Correlation between tenure and total charges
result = pearson_correlation_analysis(df, 'tenure', 'TotalCharges')
```

### 6.2 Spearman Correlation

**Purpose**: Measure monotonic relationship (not necessarily linear)

**When to Use**:
- Ordinal variables
- Non-linear relationships
- Non-normal distributions
- Outliers present

**Python Implementation**:

```python
from scipy.stats import spearmanr

def spearman_correlation_analysis(df, var1, var2, alpha=0.05):
    """
    Spearman rank correlation analysis.
    """
    print(f"\n{'='*70}")
    print(f"SPEARMAN CORRELATION: {var1} vs {var2}")
    print(f"{'='*70}\n")
    
    data = df[[var1, var2]].dropna()
    
    # Calculate Spearman correlation
    rho, p_value = spearmanr(data[var1], data[var2])
    
    print("1. Correlation Results:")
    print(f"   Spearman ρ (rho): {rho:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    # Compare with Pearson
    r_pearson, _ = pearsonr(data[var1], data[var2])
    print(f"\n2. Comparison:")
    print(f"   Pearson r:  {r_pearson:.4f}")
    print(f"   Spearman ρ: {rho:.4f}")
    print(f"   Difference: {abs(r_pearson - rho):.4f}")
    
    if abs(r_pearson - rho) > 0.1:
        print("   ⚠ Large difference suggests non-linear relationship")
    else:
        print("   ✓ Similar values suggest linear relationship")
    
    return {'rho': rho, 'p_value': p_value}

# Example
result = spearman_correlation_analysis(df, 'tenure', 'MonthlyCharges')
```

### 6.3 Point-Biserial Correlation

**Purpose**: Correlation between continuous and binary variable

**When to Use**:
- Relationship between numeric variable and Churn (binary)

**Python Implementation**:

```python
from scipy.stats import pointbiserialr

def point_biserial_analysis(df, continuous_var, binary_var, alpha=0.05):
    """
    Point-biserial correlation for continuous vs binary variable.
    """
    print(f"\n{'='*70}")
    print(f"POINT-BISERIAL CORRELATION")
    print(f"{continuous_var} vs {binary_var}")
    print(f"{'='*70}\n")
    
    # Ensure binary variable is 0/1
    data = df[[continuous_var, binary_var]].dropna()
    if data[binary_var].dtype == 'object':
        binary_map = {data[binary_var].unique()[0]: 0,
                     data[binary_var].unique()[1]: 1}
        data[binary_var] = data[binary_var].map(binary_map)
    
    # Calculate correlation
    r_pb, p_value = pointbiserialr(data[binary_var], data[continuous_var])
    
    print("1. Correlation Results:")
    print(f"   Point-biserial r: {r_pb:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    print(f"\n2. Interpretation:")
    if r_pb > 0:
        print(f"   Positive correlation: Higher {continuous_var} → More likely {binary_var}=1")
    else:
        print(f"   Negative correlation: Higher {continuous_var} → More likely {binary_var}=0")
    
    if p_value < alpha:
        print(f"\n3. Conclusion: SIGNIFICANT relationship (p < {alpha})")
    else:
        print(f"\n3. Conclusion: NO significant relationship (p ≥ {alpha})")
    
    return {'r_pb': r_pb, 'p_value': p_value}

# Example: Tenure vs Churn
result = point_biserial_analysis(df, 'tenure', 'Churn')
```

### 6.4 Correlation Matrix and Heatmap

**Purpose**: Visualize all pairwise correlations

**Python Implementation**:

```python
def comprehensive_correlation_analysis(df, method='pearson'):
    """
    Create comprehensive correlation matrix with visualization.
    """
    # Select numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Calculate correlation matrix
    if method == 'pearson':
        corr_matrix = numeric_df.corr()
    elif method == 'spearman':
        corr_matrix = numeric_df.corr(method='spearman')
    
    # Create visualization
    plt.figure(figsize=(12, 10))
    
    # Create mask for upper triangle
    mask = np.triu(np.ones_like(corr_matrix), k=1)
    
    # Plot heatmap
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', 
                center=0, fmt='.2f', square=True, linewidths=1,
                cbar_kws={"shrink": 0.8})
    
    plt.title(f'{method.capitalize()} Correlation Matrix', 
              fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Find strongest correlations with target (if Churn exists)
    if 'Churn' in corr_matrix.columns:
        print("\nStrongest Correlations with Churn:")
        churn_corr = corr_matrix['Churn'].abs().sort_values(ascending=False)
        print(churn_corr[1:11])  # Top 10, excluding Churn itself
    
    return corr_matrix

# Usage
corr_matrix = comprehensive_correlation_analysis(df, method='pearson')
```

---

## 7. Distribution Analysis

Understanding data distributions is critical for choosing appropriate statistical tests and models.

### 7.1 Normality Tests

#### 7.1.1 Shapiro-Wilk Test

**Purpose**: Test if data comes from normal distribution

**Python Implementation**:

```python
from scipy.stats import shapiro

def test_normality(data, variable_name, alpha=0.05):
    """
    Comprehensive normality testing.
    """
    print(f"\n{'='*70}")
    print(f"NORMALITY TEST: {variable_name}")
    print(f"{'='*70}\n")
    
    # Shapiro-Wilk test
    stat, p_value = shapiro(data.sample(min(5000, len(data))))  # Sample for large datasets
    
    print("1. Shapiro-Wilk Test:")
    print(f"   Statistic: {stat:.4f}")
    print(f"   p-value: {p_value:.4f}")
    
    if p_value > alpha:
        print(f"   ✓ Data appears normally distributed (p > {alpha})")
        normal = True
    else:
        print(f"   ✗ Data deviates from normal distribution (p ≤ {alpha})")
        normal = False
    
    # Visual checks
    